# to do
- get all minimal db parameters with fake data if needed
    - issue ident simbad has NAME Proxima b instead of Proxima b, so need function that cuts away 'NAME ' from those objects. use for this .lstrip("NAME") put try to keep it as one ident
- get from temp to actual once exomercat working again
- clean up code
    - get building function understandable
    - clean up exomercat confusing comments & comment internal function arguments
    - use isinstance(object, type) instead of type comparison
    - insted of specifying the type in the function header specify it in the def part e.g. def greeting(name: str) -> str: specifies input and return type
    - limit line length to 79 characters
    

In [1]:
"""
This code was written by Franziska Menti in 2022. 
It creates the tables for the prototype of the LIFE target database.
"""

#general
import numpy as np #arrays
import pyvo as vo #catalog query
import astropy as ap

In [2]:
#-------------------global helper functions-----------------------
def save(cats,paths):
    """
    This functions saves the tables given as python list in the cats parameter.
    The saving location is 'data/{path}.xml' where path is given in the paths 
    parameter.
    :param cats: Python list of astropy table to be saved.
    :param paths: Python list of paths to where to save the tables given in cats.
    """
    #go through all the elements in both lists
    for cat,path in zip(cats,paths):
        #for each column header
        for i in cat.colnames:
            #if the type of the column is object (=adaptable length string)
            if cat[i].dtype == object:
                #transform the type into string
                cat[i] = cat[i].astype(str)
        #save the table
        ap.io.votable.writeto(ap.io.votable.from_table(cat), f'data/{path}.xml')
    return

def stringtoobject(cat,number=100):
    """
    This function changes from string to object format. 
    The later has the advantace of allowing strings of varying length. 
    Without it strings can get truncated.
    :param cat: Astropy table.
    :param number: Length of longest string type element in the table. Defeault is 100.
    :return cat: Astropy table.
    """
    #defining string types as calling them string does not work and instead the type name 
    #<U3 is needed for a string of length 3
    stringtypes=[np.dtype(f'<U{j}') for j in range(1,number)]
    #for each column header
    for i in cat.colnames:
        #if the type of the column is string
        if cat[i].dtype in stringtypes:
            #transform the type into object
            cat[i] = cat[i].astype(object)
    return cat

def load(paths):
    """
    This function loads the tables saved in XML format at saving locations specified in paths.
    :param paths: Python list of saving locations.
    :return cats: Python list of loaded astropy tables.
    """
    #initialize return parameter as list
    cats=[]
    #go through all the elements in the paths list
    for path in paths:
        #read the saved data into the cats lists as astropy votable element
        cats.append(ap.io.votable.parse_single_table(f'data/{path}.xml').to_table())
    #go through all the tables in the cats list
    for cat in cats:
        cat=stringtoobject(cat,3000)
    return cats

In [3]:
#-------------------initialization function----------------------------
def initialize_database_tables():
    """
    This function initializes the database tables with no data in them.
    :return return_list: List of astropy tables in the order sources, objects,
                ident (identifiers), h_link (relation between 
                objects),star_basic,planet_basic, disk_basic,
                mesDist (distance measurements) and 
                mesMass (mass measurements). 
    """
    #id stands for identifier, idref for reference identifier
    #initialize tables with no data but column names and data type specified.
    #parameter source idref is the identifier in the source table corresponding to this parameter
    
    objects=ap.table.Table(
        #object id, type of object, all identifiers, main id
        names=['object_id','type','ids','main_id'],
        dtype=[int,object,object,object])
    
    #identifier table
    ident=ap.table.Table(
        #object idref, id, source idref for the id parameter, id reference
        names=['object_idref','id','id_source_idref','id_ref'],
        dtype=[int,object,int,object])
    
    #hierarchical link table (= relation between objects)
    h_link=ap.table.Table(
        #child object idref (e.g. planet X), 
        #parent object idref (e.g. host star of planet X)
        #source idref of h_link parameter, h_link reference, 
        #membership probability
        names=['child_object_idref','parent_object_idref',
               'h_link_source_idref','h_link_ref','membership'],
        dtype=[int,int,int,object,int])
    
    star_basic=ap.table.Table(
        #object idref, RA coordinate, DEC coordinate, 
        #coordinate error ellypse angle, major axis and minor axis,
        #coordinate quality, source idref of coordinate parameter, 
        #coordinate reference, parallax value, parallax error, parallax quality
        #source idref of parallax parameter ... same for distance parameter
        names=['object_idref','coo_ra','coo_dec','coo_err_angle',
               'coo_err_maj','coo_err_min','coo_qual',
               'coo_source_idref','coo_ref',
               'plx_value','plx_err','plx_qual','plx_source_idref',
               'plx_ref',
               'dist_value','dist_err','dist_qual','dist_source_idref',
               'dist_ref'],
        dtype=[int,float,float,float,
               float,float,object,
               int,object,
               float,float,object,int,
               object,
               float,float,object,int,
               object])
    
    planet_basic=ap.table.Table(
        #object idref, mass value, mass error, mass realtion (min, max, equal),
        #mass quality, source idref of mass parameter, mass reference
        names=['object_idref','mass_val','mass_err','mass_rel','mass_qual',
               'mass_source_idref','mass_ref'],
        dtype=[int,float,float,object,object,int,object])
    
    disk_basic=ap.table.Table(
        #object idref, black body radius value, bbr error, bbr relation (min, max, equal),
        #bbr quality,...
        names=['object_idref','rad_value','rad_err','rad_rel','rad_qual',
               'rad_source_iderf','rad_ref'],
        dtype=[int,float,float,object,object,int,object])
    
    sources=ap.table.Table(
        #reference,...
        names=['ref','provider_name','provider_url','provider_bibcode','source_id'],
        dtype=[object,object,object,object,int])
    
    mesDist=ap.table.Table(
        names=['object_idref','dist_value','dist_err','dist_qual',
               'dist_source_idref','dist_ref'],
        dtype=[int,float,float,object,
               int,object])
    
    mesMass=ap.table.Table(
        names=['object_idref','mass_val','mass_err','mass_rel','mass_qual',
               'mass_source_idref','mass_ref'],
        dtype=[int,float,float,object,object,int,object])
    
    #save all tables
    save([sources,objects,ident,h_link,star_basic,planet_basic,disk_basic,mesDist,mesMass],
         ['empty_sources','empty_objects','empty_ident','empty_h_link','empty_star_basic',
          'empty_planet_basic','empty_disk_basic','empty_mesDist','empty_mesMass'])
    return_list=[sources,objects,ident,h_link,star_basic,planet_basic,disk_basic,mesDist,mesMass]
    return return_list

In [4]:
#------------------------------provider helper functions--------------------------
def query(link,query,catalogs=[]):
    """
    Performs a query via TAP on the service given in the link parameter. 
    If a list of tables is given in the catalogs parameter, 
    those are uploaded to the service. 
    :param link: Service access URL.
    :param query: Query to be asked of the external database service in ADQL.
    :param catalogs: List of astropy tables to be uploaded to the service.
    :return cat: Astropy table containing the result of the query.
    """
    #defining the vo service using the given link
    service = vo.dal.TAPService(link)
    #without upload tables
    if catalogs==[]:
        result=service.run_async(query.format(**locals()), maxrec=160000)
    #with upload tables
    else:
        tables={}
        for i in range(len(catalogs)):
            tables.update({f"t{i+1}":catalogs[i]})
        result = service.run_async(query,uploads=tables,timeout=None, maxrec=160000)
    cat=result.to_table()
    return cat

def sources_table(cat,ref_columns,provider,old_sources=ap.table.Table()): #put this into source function
    """
    This function creates or updates the source table out of the given references. 
    The entries are unique and the columns consist out of the reference, provider_name, 
    provider_url and provider_bibcode.
    :param cat: Astropy table on which the references should be gathered.
    :param ref_columns: Header of the columns containing reference information.
    :param provider: List containing name, url and bibcode of provider.
    :param old_sources: Previously created reference table.
    :return sources: Astropy table containing references and provider information.
    """
    cat_sources=ap.table.Table() #table initialization
    cat_reflist=[] #initialization of list to store reference information
    #for all the columns given add reference information to the cat_reflist
    for k in range(len(ref_columns)):
        #In case the column has elements that are masked skip those
        if type(cat[ref_columns[k]])==ap.table.column.MaskedColumn:
            cat_reflist.extend(cat[ref_columns[k]][np.where(cat[ref_columns[k]].mask==False)])
        else:
            cat_reflist.extend(cat[ref_columns[k]])
    cat_sources['ref']=cat_reflist #add list of collected references to the table and call the column ref
    cat_sources=ap.table.unique(cat_sources)#keeps only unique values
    #attaches service information
    cat_sources['provider_name']=[provider[0]]*len(cat_sources)
    cat_sources['provider_url']=[provider[1]]*len(cat_sources)
    cat_sources['provider_bibcode']=[provider[2]]*len(cat_sources)
    #combine old and new sources into one table
    sources=ap.table.vstack([old_sources,cat_sources])
    sources=ap.table.unique(sources) #remove double entries
    return sources

def fetch_main_id(cat,colname='oid',name='main_id',oid=True):
    """
    Joins main_id from simbad to the column colname. Returns the whole
    table cat but without any rows where no simbad main_id was found.
    :param cat: Astropy table.
    :param colname: Column header of the identifiers that should be searched in SIMBAD.
    :param name: Column header for the SIMBAD main identifiers, default is main_id.
    :param oid: Specifies wether colname is a SIMBAD oid or normal identifier.
    :return cat: Astropy table with all identifiers that could be found 
        in SIMBAD and column contining ther main identifier.
    """
    #improvement idea to be performed at one point
    print('tbd option to match on position instead of main_id or oid')
    #SIMBAD TAP service
    TAP_service="http://simbad.u-strasbg.fr:80/simbad/sim-tap"
    #creating oid query
    if oid:
        main_id_query='SELECT b.main_id AS '+name+""",t1.*
                    FROM basic AS b
                    JOIN TAP_UPLOAD.t1 ON b.oid=t1."""+colname
    #creating identifier query
    else:
        main_id_query='SELECT b.main_id AS '+name+""",t1.*
                    FROM basic AS b
                    JOIN ident ON ident.oidref=b.oid
                        JOIN TAP_UPLOAD.t1 ON ident.id=t1."""+colname
    #performing query using external function
    cat=query(TAP_service,main_id_query,[cat])
    return cat

In [5]:
#-----------------------------provider data ingestion------------------
def provider_simbad():
    """
    This function obtains the SIMBAD data and arranges it in a way 
    easy to ingest into the database.
    :return sim_sources: Astropy table containing reference data.
    :return sim_objects: Astropy table containing object data.
    :return sim_ident: Astropy table containing identifier data.
    :return sim_h_link: Astropy table containing object to object relation data.
    :return sim_star_basic: Astropy table containing basic stellar data.
    :return sim_mesDist: Astropy table containing distance measurement data.
    """
    #---------------define provider------------------------
    TAP_service="http://simbad.u-strasbg.fr:80/simbad/sim-tap"
    provider_name='SIMBAD'
    provider_bibcode='2000A&AS..143....9W'
    #---------------define queries---------------------------
    select="""SELECT b.main_id,b.ra AS coo_ra,b.dec AS coo_dec,
        b.coo_err_angle, b.coo_err_maj, b.coo_err_min,b.oid, 
        b.coo_bibcode AS coo_ref, b.coo_qual,
        b.plx_err, b.plx_value, b.plx_bibcode AS plx_ref,b.plx_qual,
        h_link.membership, h_link.parent AS parent_oid, 
        h_link.link_bibcode AS h_link_ref, a.otypes,ids.ids
        """#which parameters to query from simbad and what alias to give them
    adql_query=[
        select+"""
        FROM basic AS b
        JOIN ids ON b.oid=ids.oidref
            JOIN alltypes AS a ON b.oid=a.oidref
                LEFT JOIN h_link ON b.oid=h_link.child
        WHERE b.plx_value >=50."""]
    #creating one table out of parameters from multiple ones and
    #keeping only objects with parallax bigger than 50mas
    
    upload_query=[
        #query for systems without parallax data but children (in TAP_UPLOAD.t1 table)
        #with parallax bigger than 50mas
        select+"""
        FROM basic AS b
        JOIN ids ON b.oid=ids.oidref
            JOIN alltypes AS a ON b.oid=a.oidref
                LEFT JOIN h_link ON b.oid=h_link.child
                    JOIN TAP_UPLOAD.t1 ON b.oid=t1.parent_oid
        WHERE (b.plx_value IS NULL) AND (otype='**..')""",
        #query for planets without parallax data but host star (in TAP_UPLOAD.t1 table)
        #with parallax bigger than 50mas
        select+"""
        FROM basic AS b
        JOIN ids ON b.oid=ids.oidref
            JOIN alltypes AS a ON b.oid=a.oidref
                LEFT JOIN h_link ON b.oid=h_link.child
                    JOIN TAP_UPLOAD.t1 ON b.oid=t1.oid
        WHERE (b.plx_value IS NULL) AND (otype='Pl..')""",
        #query all distance measurements for objects in TAP_UPLOAD.t1 table
        """SELECT oid, dist AS dist_value, plus_err, qual AS dist_qual,
        bibcode AS dist_ref,minus_err,dist_prec
        FROM mesDistance
        JOIN TAP_UPLOAD.t1 ON oidref=t1.oid""",
        #query all identifiers for objects in TAP_UPLOAD.t1 table
        """SELECT id, t1.*
        FROM ident 
        JOIN TAP_UPLOAD.t1 ON oidref=t1.oid"""]
    #define header name of columns containing references data
    ref_columns=[['coo_ref','plx_ref'],['h_link_ref'],['dist_ref'],['id_ref']]
    #------------------querrying------------------------
    #perform query for objects with parallax >50mas
    simbad=query(TAP_service,adql_query[0])
    #querries parent and children objects with no parallax value
    parents_without_plx=query(TAP_service,upload_query[0],[simbad])
    children_without_plx=query(TAP_service,upload_query[1],[simbad])
    #adding of no_parallax objects to rest of simbad query objects
    simbad=ap.table.vstack([simbad,parents_without_plx])
    simbad=ap.table.vstack([simbad,children_without_plx])
    #----------------------sorting object types------------------------
    #sorting from object type into star, system and planet type
    simbad['type']=['None' for i in range(len(simbad))]
    simbad['multiple']=[False for i in range(len(simbad))]
    to_remove_list=[]
    for i in range(len(simbad)):
        #planets
        if "Pl" in simbad['otypes'][i]:
            simbad['type'][i]='pl'
        #stars
        elif "*" in simbad['otypes'][i]:
            #system containing multiple stars
            if "**" in simbad['otypes'][i]:
                simbad['type'][i]='sy'
                simbad['multiple'][i]=True
            #individual stars
            else:
                simbad['type'][i]='st'
        else:
            #tell me if you found any objects that are neither star, system nor planet
            #most likely single brown dwarfs
            print('Removed object because type neither Pl,* or **:',simbad['otypes'][i])
            #storing information for later removal from table called simbad
            to_remove_list.append(i)
    #removing any objects that are neither planet, star or system in type
    if to_remove_list!=[]:
        simbad.remove_rows(to_remove_list)
    
    #creating helpter table stars
    temp_stars=simbad[np.where(simbad['type']!='pl')]
    #removing double objects (in there due to multiple parents)
    stars=ap.table.Table(ap.table.unique(temp_stars,keys='main_id'),copy=True)
    
    #-----------------creating output table sim_ident------------------------------
    sim_ident=query(TAP_service,upload_query[3],[simbad['oid','main_id'][:].copy()])
    sim_ident['id_ref']=[provider_bibcode for j in range(len(sim_ident))]
    sim_ident.remove_column('oid')        
    
    #-------------------creating output table sim_mesDist---------------------
    sim_mesDist=query(TAP_service,upload_query[2],[stars[:].copy()])
    print(sim_mesDist['dist_value'].fill_value)
    print('insert here something to fill in null values that make more sense than like nan')
    print('so here it works but not further below, maybe do it in the building as maybe saving is issue')
    sim_mesDist=fetch_main_id(sim_mesDist)
    sim_mesDist['dist_err']=np.maximum(sim_mesDist['plus_err'],-sim_mesDist['minus_err'])
    sim_mesDist.remove_rows(sim_mesDist['dist_err'].mask.nonzero()[0])
    #group by oid
    grouped_mesDist=sim_mesDist.group_by('main_id')
    best_mesDist=sim_mesDist['main_id','dist_value','plus_err','dist_qual','dist_ref'][:0]
    best_mesDist.rename_column('plus_err','dist_err')
    for i in range(len(grouped_mesDist.groups.keys)):
        #sort by quality
        row=grouped_mesDist.groups[i][np.where(grouped_mesDist['dist_prec'].groups[i]==np.max(grouped_mesDist['dist_prec'].groups[i]))][0]
        #take first and add to best_paras
        #which error to take when there are multiples...
        best_mesDist.add_row([row['main_id'],row['dist_value'], row['dist_err'],row['dist_qual'], row['dist_ref']])
    #join with other multimes thingis
    best_paras=best_mesDist#TBD when more multi measurement tables are implemented: vstack them here
    sim_mesDist=sim_mesDist['main_id','dist_value','dist_err','dist_qual','dist_ref']
    
    #--------------------creating helper table sim_stars---------------------
    #add best para from multiple measurements tables
    stars=ap.table.join(stars,best_paras,keys='main_id',join_type='left')
    
    #--------------creating output table sim_h_link ---------------
    sim_h_link=simbad['main_id','parent_oid','h_link_ref','membership']
    #if you want to exclude objects with lower membership probability use this line instead:
    #sim_h_link=simbad['main_id','parent_oid','h_link_ref','membership'][np.where(simbad['membership']>50)]
    #consequence is that you loose objects with no membership value given (~) e.g. alf cen system
    print('all membership values included, use commented out code to change')
    sim_h_link=fetch_main_id(sim_h_link,'parent_oid','parent_main_id')
    sim_h_link.remove_column('parent_oid')
    #null values
    sim_h_link['membership'].fill_value=-1
    sim_h_link['membership']=sim_h_link['membership'].filled()
    #-----------------creating output table sim_planets-----------------------------
    temp_sim_planets=simbad['main_id','ids','type'][np.where(simbad['type']=='pl')]
    sim_planets=ap.table.Table(ap.table.unique(temp_sim_planets,keys='main_id'),copy=True)
    #-----------------creating output table sim_objects-------------------------------
    sim_objects=ap.table.vstack([sim_planets['main_id','ids','type'],
                             stars['main_id','ids','type']])
    sim_objects['ids']=sim_objects['ids'].astype(object)
    #--------------creating output table sim_sources ---------------
    sim_sources=ap.table.Table()
    tables=[stars, sim_h_link, sim_mesDist,sim_ident]
    for cat,ref in zip(tables,ref_columns):
        sim_sources=sources_table(cat,ref,[provider_name,TAP_service,
                                           provider_bibcode],sim_sources)
    #------------------------creating output table sim_star_basic------------------------------
    sim_star_basic=stars['main_id','coo_ra','coo_dec','coo_err_angle','coo_err_maj',
                         'coo_err_min','coo_qual','coo_ref',
                         'plx_value','plx_err','plx_qual','plx_ref',
                        'dist_value','dist_err','dist_qual','dist_ref']
    save([sim_sources,sim_objects,sim_ident,sim_h_link,sim_star_basic,sim_mesDist],
         ['sim_sources','sim_objects','sim_ident','sim_h_link','sim_star_basic',
          'sim_mesDist'])
    return sim_sources,sim_objects,sim_ident,sim_h_link,sim_star_basic,sim_mesDist

def provider_gk():
    """
    This function obtains the disk data and arranges it in a way 
    easy to ingest into the database.
    :return gk_sources: Astropy table containing reference data.
    :return gk_objects: Astropy table containing object data.
    :return gk_ident: Astropy table containing identifier data.
    :return gk_h_link: Astropy table containing object to object relation data.
    :return gk_disk_basic: Astropy table containing basic disk data.
    """
    #---------------define provider------------------------
    provider_name='priv. comm.'
    TAP_service='None'
    provider_bibcode='None'
    #loading table obtained via direct communication from Grant Kennedy
    gk_disks=ap.io.votable.parse_single_table(
        "data/Grant_absil_2013_.xml").to_table()
    #transforming from string type into object to have variable length
    gk_disks=stringtoobject(gk_disks,212)
    #removing objects with plx_value=='None'
    gk_disks=gk_disks[np.where(gk_disks['plx_value']!='None')]
    #converting masked plx_value into -99
    gk_disks['plx_value'].fill_value=-99
    gk_disks['plx_value']=gk_disks['plx_value'].astype(float)
    #sorting out everything with plx_value not >50 (corresponding to 50mas=20pc)
    gk_disks=gk_disks[np.where(gk_disks['plx_value']>50.)]
    #adds the column for object type 
    gk_disks['type']=['di' for j in range(len(gk_disks))]
    gk_disks['disks_ref']=['Grant Kennedy' 
                        for j in range(len(gk_disks))]
    #making sure identifiers are unique
    ind=gk_disks.group_by('id').groups.indices
    for i in range(len(ind)-1):
        l=ind[i+1]-ind[i]
        if l==2:
            gk_disks['id'][ind[i]]=gk_disks['id'][ind[i]]+'a'
            gk_disks['id'][ind[i]+1]=gk_disks['id'][ind[i]+1]+'b'
        if l>2:
            print('more than two disks with same name')
    #fetching updated main identifier of host star from simbad
    gk_disks.rename_column('main_id','gk_host_main_id')
    gk_disks=fetch_main_id(gk_disks,colname='gk_host_main_id',name='main_id',oid=False)
    #--------------creating output table gk_h_link ---------------
    gk_h_link=gk_disks['id','main_id','disks_ref']
    gk_h_link.rename_columns(['main_id','disks_ref'],['parent_main_id','h_link_ref'])
    gk_h_link.rename_column('id','main_id')
    #--------------creating output table gk_objects ---------------
    gk_disks['ids']=gk_disks['id']#because only single id per source given
    gk_objects=gk_disks['id','ids','type']
    gk_objects.rename_column('id','main_id')
    #--------------creating output table gk_ident ---------------
    gk_ident=gk_disks['ids','id','disks_ref']
    #actually would want to use id instad of ids but gets error and ids is same column as id
    gk_ident.rename_columns(['ids','disks_ref'],['main_id','id_ref'])
    #--------------creating output table gk_sources ---------------
    gk_sources=sources_table(gk_disks,['disks_ref'],[provider_name,TAP_service,
                                           provider_bibcode])
    #--------------creating output table gk_disk_basic----------------
    gk_disk_basic=gk_disks['id','rdisk_bb','e_rdisk_bb','disks_ref']
    #converting from string to float
    for column in ['rdisk_bb','e_rdisk_bb']:
        #replacing 'None' with 'nan' as the first one is not float convertible
        temp_length=len(gk_disk_basic[column][np.where(gk_disk_basic[column]=='None')])
        gk_disk_basic[column][np.where(gk_disk_basic[column]=='None')]=['nan' for i in range(temp_length)]
        gk_disk_basic[column].fill_value='nan'
        gk_disk_basic[column].filled()
        gk_disk_basic[column]=gk_disk_basic[column].astype(float)
    gk_disk_basic.rename_columns(['id','rdisk_bb','e_rdisk_bb','disks_ref'],
                                 ['main_id','rad_value','rad_err','rad_ref'])
    save([gk_sources,gk_objects, gk_ident, gk_h_link,gk_disk_basic],
         ['gk_sources','gk_objects', 'gk_ident', 'gk_h_link','gk_disk_basic'])
    return gk_sources,gk_objects, gk_ident, gk_h_link,gk_disk_basic

def provider_exo(temp=True):
    """
    This function obtains the exomercat data and arranges it in a way 
    easy to ingest into the database. Currently the exomercat server is not online.
    A temporary method to ingest old exomercat data was implemented and can be accessed
    by setting temp=True as argument.
    :return exo_sources: Astropy table containing reference data.
    :return exo_objects: Astropy table containing object data.
    :return exo_ident: Astropy table containing identifier data.
    :return exo_h_link: Astropy table containing object to object relation data.
    :return exo_planet_basic: Astropy table containing basic planetary data.
    :return exo_mesMass: Astropy table containing mass measurement data.
    """
    #---------------define provider------------------------
    TAP_service="http://archives.ia2.inaf.it/vo/tap/projects"
    provider_name='Exo-MerCat'
    provider_bibcode='2020A&C....3100370A'
    #---------------define query---------------------------
    adql_query="""SELECT *
                  FROM exomercat.exomercat"""
    #---------------obtain data----------------------
    if temp:
        #exomercat=ap.io.votable.parse_single_table("data/raw_exomercat.xml").to_table()
        exomercat=ap.io.ascii.read("data/exomercat_Sep2.csv")
        exomercat=stringtoobject(exomercat,3000)

    else:
        exomercat=query(TAP_service,adql_query)
    #----------------putting object main identifiers together-------------------
    #for planet and host for later comparison of objects from different sources    
    exomercat['planet_main_id']=ap.table.Column(dtype=object, length=len(exomercat))#initializing column
    exomercat['host_main_id']=exomercat['main_id']
    for i in range(len(exomercat)):
        if type(exomercat['main_id'][i])!=np.ma.core.MaskedConstant:
            hostname=exomercat['main_id'][i]
        else:
            hostname=exomercat['host'][i]
        if type(exomercat['binary'][i])!=np.ma.core.MaskedConstant:
            exomercat['host_main_id'][i]=hostname+' '+exomercat['binary'][i]
        else:
            exomercat['host_main_id'][i]=hostname
        exomercat['planet_main_id'][i]=exomercat['host_main_id'][i]+' '+exomercat['letter'][i]

    def sort_out_20pc(cat,colnames):
        """
        This Function sorts out objects not within 20pc. The value comes from the 
        LIFE database distance cut.
        """
        [sim_objects]=load(['sim_objects'])
        sim_objects.rename_column('main_id','temp')
        cat_old=ap.table.Table()
        for colname in colnames:
            cat=ap.table.join(cat,sim_objects['temp','ids'],keys_left=colname,keys_right='temp')
            cat.remove_columns(['temp','ids'])
            cat_old=ap.table.vstack([cat,cat_old])
        cat=ap.table.unique(cat_old,silent=True)
        return cat
        
    exo=exomercat
    exomercat=sort_out_20pc(exomercat,['host_main_id'])

    #removing whitespace in front of main_id and name. 
    #if it were done before sort_out_20pc issue with missing values would occur 
    for i in range(len(exomercat)):
        exomercat['planet_main_id'][i]=exomercat['planet_main_id'][i].strip()
        exomercat['main_id'][i]=exomercat['main_id'][i].strip()
        exomercat['name'][i]=exomercat['name'][i].strip()
    #join exomercat on host_main_id and sim_objects main_id
    #using simbad instead of other cataloges to determine if a star is within 20 pc will mean I loose some of them.
    #that is, however, preferrable to having to do the work of checking the literature.
    #a compromise is to keep the list of objects I lost.
    
    #show which elements from exomercat were not found in sim_objects
    exo['name']=exo['name'].astype(object)
    removed_objects=ap.table.setdiff(exo,exomercat,keys=['name'])
    print('length exomercat before join',len(exo['host_main_id']))
    print('length exomercat after join on host_main_id',len(exomercat['host_main_id']))
    print('number of removed_objects',len(removed_objects['host_main_id']))###----------------hm 7000 there, so join didn't work----------
    save([removed_objects],['exomercat_removed_objects'])
    print('tbd: improve to not loose that many objects') 
    
    #-------------exo_ident---------------
    #['main_id','id','id_ref']
    exo_ident=exomercat['planet_main_id','name']
    exo_ident.rename_columns(['planet_main_id','name'],['main_id','id'])
    for i in range(len(exomercat)):
        if exomercat['planet_main_id'][i]!=exomercat['name'][i]:
            exo_ident.add_row([exomercat['planet_main_id'][i],exomercat['planet_main_id'][i]])
    exo_ident['id_ref']=[provider_bibcode for j in range(len(exo_ident))]
    print(exo_ident[np.where(exo_ident['main_id']=='Wolf  940 b')])
    print('I have a wrong double object in exo_ident because there are different amount of white spaces between catalog and number')
    #-------------exo_objects---------------
    print('tbd at one point: I think I want to add hosts to object')
    exo_objects=ap.table.Table(names=['main_id','ids'],dtype=[object,object])
    grouped_exo_ident=exo_ident.group_by('main_id')
    ind=grouped_exo_ident.groups.indices
    for i in range(len(ind)-1):#-1 is needed because else ind[i+1] is out of bonds
        ids=[]
        for j in range(ind[i],ind[i+1]):
            ids.append(grouped_exo_ident['id'][j])
        ids="|".join(ids)
        exo_objects.add_row([grouped_exo_ident['main_id'][ind[i]],ids])
    exo_objects['type']=['pl' for j in range(len(exo_objects))]

    #-------------------exo_mesMass---------------------
    exomercat['mass_max'].fill_value=999
    exomercat['mass_min'].fill_value=-999
    exomercat['mass_max']=exomercat['mass_max'].filled()
    exomercat['mass_min']=exomercat['mass_min'].filled()
    exomercat['mass_err']=np.maximum(exomercat['mass_max'],-exomercat['mass_min'])
    exo_mesMass=exomercat['planet_main_id','mass','mass_err','mass_url']
    exo_mesMass.rename_columns(['planet_main_id','mass','mass_url'],
                                    ['main_id','mass_val','mass_ref'])
    print('issue is that I have many null value ones (null expr is 1w+20 for val and 999 for err) in here but should only contain those with actual measurements')
    #exo_mesMass.remove_rows(exo_mesMass[np.where(mesMass['mass_val']==1e+20)])
    
    grouped_mesMass=exo_mesMass.group_by('main_id')
    #____ ahm here, dont forget to delete some stuff below
    best_mesMass=exo_mesMass['main_id','mass_val','mass_err','mass_ref'][:0]
    for i in range(len(grouped_mesMass.groups.keys)):
        #sort by quality
        row=grouped_mesMass.groups[i][np.where(grouped_mesMass['mass_err'].groups[i]==np.min(grouped_mesMass['mass_err'].groups[i]))][0]
        #take first and add to best_paras
        #which error to take when there are multiples...
        best_mesMass.add_row([row['main_id'],row['mass_val'], row['mass_err'],row['mass_ref']])
    #join with other multimes thingis
    best_paras=best_mesMass#vstack other multi meas tables
    
    #some of the stars are not in simbad
    #-------------exo_h_link---------------
    #['child_object_idref','parent_object_idref',
               #'h_link_source_idref','h_link_ref','membership']
    exo_h_link=exomercat['planet_main_id', 'host_main_id']
    exo_h_link.rename_columns(['planet_main_id','host_main_id'],
                              ['main_id','parent_main_id'])
    exo_h_link['h_link_ref']=[provider_bibcode for j in range(len(exo_h_link))]

    #I decided not to take catalog as h_link_ref, as it would introduce 16 different sources when it
    #just means providers of provider.
    #the correct way would be to add the four different databases as sources and have membership from
    #status string column
    #this, however, is more detail than needed for just prototype database
    #-------------exo_planet_basic
    #exchanged bestmass with mass because had issues of different references for same value and object
    #did not solve issue, need to create like for simbad special table for mass values and then only
    #add single ones to planet_basic
    exo_planet_basic=best_mesMass
    
    #-------------exo_sources---------------
    ref_columns=[['mass_url'],['h_link_ref'],['id_ref']]
    exo_sources=ap.table.Table()
    tables=[exomercat, exo_h_link,exo_ident]
    for cat,ref in zip(tables,ref_columns):
        exo_sources=sources_table(cat,ref,[provider_name,TAP_service,
                                           provider_bibcode],exo_sources) 
        
    #print('to be saved:',[exo_sources,exo_objects,exo_ident,exo_h_link,exo_planet_basic])

    save([exo_sources,exo_objects,exo_ident,exo_h_link,exo_planet_basic,exo_mesMass],
         ['exo_sources','exo_objects','exo_ident','exo_h_link','exo_planet_basic','exo_mesMass'])
        
    return exo_sources,exo_objects,exo_ident,exo_h_link,exo_planet_basic,exo_mesMass

In [6]:
#------------------------provider combining-----------------
def building(sim,gk,exo,temp=False):
    """
    This function builds from the input parameters the tables
    for the LIFE database.
    """
    #creates empty tables as needed for final database ingestion
    init=initialize_database_tables()
    
    #initializes 8 table objects
    #corresponding to 'sources','objects','ident','h_link','star_basic','planet_basic','disk_basic','mesDist'
    cat=[ap.table.Table() for i in range(9)]
    
    #for the sources and objects stacks tables from different providers keeping only unique values
    #then create identifiers for those tables
    
    cat[0]=ap.table.vstack([init[0],sim[0]])
    cat[0]=ap.table.vstack([cat[0],gk[0]])
    cat[0]=ap.table.vstack([cat[0],exo[0]])
    if len(cat[0])>0:
        cat[0]=ap.table.unique(cat[0],silent=True)
        cat[0]['source_id']=[j+1 for j in range(len(cat[0]))]
    
    def idsjoin(cat,column_ids1,column_ids2):
        """
        This function merges the identifiers from two different columns into one.
        """
        cat['ids']=ap.table.Column(dtype=object, length=len(cat))#initializing column
        for column in [column_ids1,column_ids2]:
            if type(cat[column])==ap.table.column.MaskedColumn:
                cat[column].fill_value=''
                cat[column]=cat[column].filled()
        for i in range(len(cat)):
            ids1=cat[column_ids1][i].split('|')#splitting object into list of elements
            ids2=cat[column_ids2][i].split('|')
            if ids2==['']:
                cat['ids'][i]=cat[column_ids1][i]
            if ids1==['']:
                cat['ids'][i]=cat[column_ids2][i]
            else:
                ids=ids1+ids2#should be list
                #removing double entries
                ids=set(ids)
                #changing type back into list
                ids=list(ids)
                #joining list into object with elements separated by |
                ids="|".join(ids)
                cat['ids'][i]=ids
        return cat
    #cat[1]=ap.table.vstack([init[1],sim[1]]) 
    cat[1]=sim[1]#removed vstack with init to not have object_id as is empty anyways
    cat[1]=ap.table.join(cat[1],gk[1],keys='main_id',join_type='outer')

    print('getting warning about column type mergeing of string types')
    print('ok could not solve the warning message about type merging in type part but stuff seems generally to work')

    def objectmerging(cat):
        cat=idsjoin(cat,'ids_1','ids_2')
        #merging types
        cat['type']=ap.table.Column(dtype=object, length=len(cat))#initializing column
        cat['type_1']=cat['type_1'].astype(object)
        cat['type_2']=cat['type_2'].astype(object)
        for i in range(len(cat)):
            if type(cat['type_2'][i])==np.ma.core.MaskedConstant:
                cat['type'][i]=cat['type_1'][i]
            else:
                cat['type'][i]=cat['type_2'][i]
        cat.remove_columns(['ids_1','ids_2','type_1','type_2'])
        return cat

    cat[1]=objectmerging(cat[1])
    cat[1]=ap.table.join(cat[1],exo[1],keys='main_id',join_type='outer')
    cat[1]=objectmerging(cat[1])
    #assigning object_id
    cat[1]['object_id']=[j+1 for j in range(len(cat[1]))]
    
    print('At one point I would like to be able to merge objects with main_id NAME Proxima Centauri b and Proxima Centauri b')
    print('should work with simbad upload')        
    #now get source_idref here
    def match(cat,sources,paras,provider):
        """
        This function joins the source identifiers to the in paras specified parameters of cat.
        """
        #for all parameters specified
        for para in paras:
            #if they have reference columns
            if para+'_ref' in cat.colnames:
                #if those reference columns are masked
                if type(cat[para+'_ref'])==ap.table.column.MaskedColumn:
                    cat[para+'_ref'].fill_value=''
                    cat=cat.filled()
                #join to each reference parameter its source_id
                cat=ap.table.join(cat,
                    sources['ref','source_id'][np.where(sources['provider_name']==provider)],
                    keys_left=para+'_ref',keys_right='ref',join_type='left')
                #renaming column to specify to which parameter the source_id correspond
                cat.rename_column('source_id',f'{para}_source_idref')
                #deleting double column containing reference information
                cat.remove_columns('ref')
                #in case the para_value entry is masked this if environment will put the source_id entry to null
                if para+'_value' in cat.colnames:
                    if type(cat[para+'_value'])==ap.table.column.MaskedColumn:
                        for i in cat[para+'_value'].mask.nonzero()[0]:
                            cat[f'{para}_source_idref'][i]=0
        return cat
    paras=[['id'],['h_link'],['coo','plx','dist'],['mass'],['rad'],['dist'],['mass']]
    
    for i in range(2,9):
        #replacing ref with source_idref columns
        #q for markus: do I need to do this or can dachs do that for me?
        #I mean creating an identifier for the refs
        
        #getting source_idref to each ref
        sim[i]=match(sim[i],cat[0],paras[i-2],'SIMBAD')
        gk[i]=match(gk[i],cat[0],paras[i-2],'priv. comm.')
        exo[i]=match(exo[i],cat[0],paras[i-2],'Exo-MerCat') 
        
        #joining data from different providers
        cat[i]=ap.table.vstack([init[i],sim[i]])#I do this to get those columns that are empty in the data
        cat[i]=ap.table.vstack([cat[i],gk[i]])
        cat[i]=ap.table.vstack([cat[i],exo[i]])
        
        #if resulting catalog not empty
        if len(cat[i])>0:
            #only keeping unique entries
            cat[i]=ap.table.unique(cat[i],silent=True)
        #i==2#--------------------ident--------------------------
        #no special action required

            if i==3:#--------------------h_link--------------------------
                #expanding from child_main_id to object_idref
                #first remove the child_object_idref we got from empty initialization
                #yes there should be a more elegant way to do this but at least it works
                cat[i].remove_column('child_object_idref')
                cat[i]=ap.table.join(cat[i],cat[1]['object_id','main_id'],keys='main_id',
                   join_type='left')
                cat[i].rename_columns(['object_id','main_id'],
                                      ['child_object_idref','child_main_id'])
                
                #expanding from parent_main_id to parent_object_idref
                cat[i].remove_column('parent_object_idref')
                #kick out any h_link rows where parent_main_id not in objects (e.g. clusters)
                cat[i]=ap.table.join(cat[i],cat[1]['object_id','main_id'],
                   keys_left='parent_main_id',keys_right='main_id')
                cat[i].remove_column('main_id')#removing because same as parent_main_id
                cat[i].rename_column('object_id','parent_object_idref')
                #null values
                cat[i]['membership'].fill_value=-1
                cat[i]['membership']=cat[i]['membership'].filled()
            
            #for all the other tables add object_idref
            else:
                #first remove the object_idref we got from empty initialization
                #yes there should be a more elegant way to do this but at least it works
                cat[i].remove_column('object_idref')
                cat[i]=ap.table.join(cat[i],cat[1]['object_id','main_id'],join_type='left')
                cat[i].rename_column('object_id','object_idref')
            if i==4:#--------------------star_basic--------------------------
                #choosing all objects with type star or system
                #I am just adding main_id because I have not found out how to do join with just one column table
                stars=cat[1]['object_id','main_id'][np.where(cat[1]['type']=='st')]
                systems=cat[1]['object_id','main_id'][np.where(cat[1]['type']=='sys')]
                temp=ap.table.vstack([stars,systems])
                temp.rename_columns(['object_id','main_id'],['object_idref','temp'])
                cat[i]=ap.table.join(cat[i],temp,join_type='outer',keys='object_idref')
                cat[i].remove_column('temp')
            if i==5:#--------------------planet_basic--------------------------
                temp=cat[1]['object_id','main_id'][np.where(cat[1]['type']=='pl')]
                temp.rename_columns(['object_id','main_id'],['object_idref','temp'])
                cat[i]=ap.table.join(cat[i],temp,keys='object_idref',join_type='outer')
                cat[i].remove_column('temp')
        else:
            print('error: empty table')
    if temp:
        cat[5]=ap.table.unique(cat[5],keys='object_idref')
    #add missing data information to basic
    #pl=objects[np.where(objects['type']=='pl')]
    #di=objects[np.where(objects['type']=='di')]
    #st=objects-pl-di
    #if len(star_basic)< len(st):
        #for obj in st['main_id']:
            #if obj not in star_basic['main_id']:
                #star_basic.add_row()
    save(cat,
         ['sources','objects','ident','h_link','star_basic','planet_basic','disk_basic','mesDist','mesMass'])
    return cat

# initialize

In [7]:
sources,objects,ident,h_link,star_basic,planet_basic,disk_basic,mesDist,mesMass=initialize_database_tables()

In [8]:
save([sources,objects,ident,h_link,star_basic,planet_basic,disk_basic,mesDist,mesMass],
         ['sources','objects','ident','h_link','star_basic',
          'planet_basic','disk_basic','mesDist','mesMass'])

# get source data

In [9]:
sim_sources,sim_objects,sim_ident,sim_h_link,sim_star_basic,sim_mesDist=provider_simbad()

Removed object because type neither Pl,* or **: BD?|IR


1e+20
insert here something to fill in null values that make more sense than like nan
so here it works but not further below, maybe do it in the building as maybe saving is issue
tbd option to match on position instead of main_id or oid


all membership values included, use commented out code to change
tbd option to match on position instead of main_id or oid


In [10]:
gk_sources,gk_objects, gk_ident, gk_h_link,gk_disk_basic=provider_gk()

tbd option to match on position instead of main_id or oid


In [11]:
exo_sources,exo_objects,exo_ident,exo_h_link,exo_planet_basic,exo_mesMass=provider_exo()

length exomercat before join 7827
length exomercat after join on host_main_id 267
number of removed_objects 7560
tbd: improve to not loose that many objects
  main_id        id            id_ref      
----------- ----------- -------------------
Wolf  940 b  Wolf 940 b 2020A&C....3100370A
Wolf  940 b Wolf  940 b 2020A&C....3100370A
I have a wrong double object in exo_ident because there are different amount of white spaces between catalog and number
tbd at one point: I think I want to add hosts to object
issue is that I have many null value ones (null expr is 1w+20 for val and 999 for err) in here but should only contain those with actual measurements


# building db

In [12]:
empty=ap.table.Table()
sim=[sim_sources,sim_objects,sim_ident,sim_h_link,sim_star_basic,empty[:],empty[:],sim_mesDist,empty[:]]
gk=[gk_sources,gk_objects, gk_ident, gk_h_link,empty[:],empty[:],gk_disk_basic,empty[:],empty[:]]
exo=[exo_sources,exo_objects, exo_ident, exo_h_link,empty[:],exo_planet_basic,empty[:],empty[:],exo_mesMass]

sources,objects,ident,h_link,star_basic,planet_basic,disk_basic,mesDist,mesMass=building(sim,gk,exo)

getting warning about column type mergeing of string types
ok could not solve the warning message about type merging in type part but stuff seems generally to work
At one point I would like to be able to merge objects with main_id NAME Proxima Centauri b and Proxima Centauri b
should work with simbad upload


# testing

In [3]:
#just loading tables
load(['sim_sources'])

[<Table length=225>
         ref         provider_name ...   provider_bibcode 
        object           object    ...        object      
 ------------------- ------------- ... -------------------
                            SIMBAD ... 2000A&AS..143....9W
 1980AJ.....85..454H        SIMBAD ... 2000A&AS..143....9W
 1986MNRAS.223..629M        SIMBAD ... 2000A&AS..143....9W
 1988AJ.....95.1841G        SIMBAD ... 2000A&AS..143....9W
 1988IRASP.C......0J        SIMBAD ... 2000A&AS..143....9W
 1991AJ....101.2220B        SIMBAD ... 2000A&AS..143....9W
                 ...           ... ...                 ...
 2021AJ....161...86D        SIMBAD ... 2000A&AS..143....9W
 2021AJ....162...11L        SIMBAD ... 2000A&AS..143....9W
 2021ApJ...909L...6P        SIMBAD ... 2000A&AS..143....9W
 2021ApJ...911....7Z        SIMBAD ... 2000A&AS..143....9W
 2022AJ....163..151S        SIMBAD ... 2000A&AS..143....9W
 2022AJ....163..168W        SIMBAD ... 2000A&AS..143....9W
 2022MNRAS.509.2091M        SIMBAD .

In [81]:
print(ident['main_id'][np.where(ident['id']=='*  14 Her b')])
print(mesMass['mass_val'][np.where(mesMass['mass_val']==1e+20)])
print(mesMass)
#current null values 1e+20 and 999.0

  main_id  
-----------
*  14 Her b
*  14 Her b
mass_val
--------
   1e+20
   1e+20
   1e+20
   1e+20
   1e+20
   1e+20
   1e+20
     ...
   1e+20
   1e+20
   1e+20
   1e+20
   1e+20
   1e+20
   1e+20
Length = 110 rows
      mass_val       mass_err mass_rel ...   main_id   object_idref
-------------------- -------- -------- ... ----------- ------------
               1e+20    999.0       -- ... *  14 Her b           10
  7.6789999999999985    999.0       -- ... *  14 Her c           11
                61.5      6.5       -- ... *  15 Sge b           14
               1e+20    999.0       -- ... *  47 UMa b           33
               1e+20    999.0       -- ... *  47 UMa c           34
               1e+20    999.0       -- ... *  47 UMa d           35
                0.46     0.01       -- ... *  51 Peg b           37
                 ...      ...      ... ...         ...          ...
               1e+20    999.0       -- ... Wolf  359 b         3502
               1e+20    999.0    

In [89]:
cat=mesDist
print(cat)
print(cat[0])
print(cat.info)
#star_basic has masked entries only in dist_source_idref
#mesDist has dist_err masked entries but not shown in mesDist.info under n_bad
#wait what with objects that dont have same main id but are part of ids of the other? could do first positional ckeck and afterwards an ids check. not urgent, do first dachs
for col in cat.colnames:
    if type(cat[col])==ap.table.column.MaskedColumn:
        print(cat[col].fill_value)

dist_value dist_err dist_qual ... minus_err dist_prec object_idref
---------- -------- --------- ... --------- --------- ------------
    18.886       --           ...   -0.0342         3            1
   17.9898       --           ...   -0.1076         4            1
   17.5652       --           ...   -0.0225         4            3
   17.5565       --           ...   -0.0099         4            3
   13.9204       --           ...   -0.0291         4            4
    13.719       --           ...    -0.147         3            4
     18.74       --           ...   -0.2841         2            5
       ...      ...       ... ...       ...       ...          ...
   18.1525       --           ...   -0.0326         4         3573
   18.1566       --           ...    -0.015         4         3573
      18.8       --           ...     1e+20         2         3573
      23.5       --           ...     1e+20         1         3573
   19.5702       --           ...   -0.0208         4         

In [638]:
#huch now I have 555 planets?? but from exomercat I have only 267, are rest from simbad? yes 286 but most likely quite some double.

#is the difference between adql join ... on ... and join ... using() that in the later case both colname are identical?


In [25]:
#issue of * alf Cen A not in h_link as child of * alf Cen -> solved by including objects from simbad with any membership
temp=ident[np.where(ident['id']=='* alf Cen')]
children=h_link[np.where(h_link['parent_object_idref']==temp['object_idref'])]
print(children)

h_link_source_idref      h_link_ref     ... parent_object_idref
                                        ...                    
------------------- ------------------- ... -------------------
                 15 2001AJ....122.3466M ...                  81
                 15 2001AJ....122.3466M ...                  81


In [26]:
#issue of h_link.colnames includes type & ids -> solved by changing join from left to normal and deleting not any longer needed last join
print(h_link.colnames)


['h_link_source_idref', 'h_link_ref', 'membership', 'parent_main_id', 'child_main_id', 'child_object_idref', 'parent_object_idref']


In [275]:
n=star_basic['dist_source_idref'].mask.nonzero()[0]
print(star_basic['main_id','dist_value','dist_ref'][n])

         main_id          dist_value dist_ref
------------------------- ---------- --------
                *   9 Pup      1e+20        ?
                *  61 Cyg      1e+20        ?
                *  70 Oph      1e+20        ?
                *  83 Leo      1e+20        ?
                * 212 Pup      1e+20        ?
                * alf Aql      1e+20        ?
                * alf Boo      1e+20        ?
                      ...        ...      ...
 WISE J045746.08-020719.2      1e+20        ?
 WISE J105257.95-194250.2      1e+20        ?
 WISE J114156.71-332635.8      1e+20        ?
 WISE J163348.95-680851.6      1e+20        ?
WISEA J085510.74-071442.5      1e+20        ?
                Wolf   66      1e+20        ?
                Wolf 1561      1e+20        ?
Length = 159 rows


In [33]:
unq, unq_idx, unq_cnt=np.unique(planet_basic['object_idref'], return_inverse=True, return_counts=True)

In [34]:
cnt_mask = unq_cnt > 1
dup_ids = unq[cnt_mask]
print(dup_ids)
#hm this sais is unique
#print(planet_basic['object_idref'])
#maybe I have masked elements?
#ok I got how to get duplicated elements
print(planet_basic[np.where(planet_basic['object_idref']==62)])
#issue is, that I have different mass references, so I will just take only mass not bestmass for now

object_idref
------------
          61
          62
mass_val mass_err mass_rel mass_qual ... mass_ref    main_id    object_idref
-------- -------- -------- --------- ... -------- ------------- ------------
    0.36     0.02       --        -- ...       EU *  83 Leo B c           62
   1e+20    999.0       --        -- ...      OEC *  83 Leo B c           62


In [27]:
exomercat=ap.io.ascii.read("data/exomercat_Sep2.csv")

In [28]:
exomercat.info

<Table length=7827>
       name         dtype     class     n_bad
------------------ ------- ------------ -----
                id   int64       Column     0
              name   str31       Column     0
              host   str26       Column     0
            letter    str1       Column     0
            ra_off float64       Column     0
           dec_off float64       Column     0
              mass float64 MaskedColumn  6148
          mass_max float64 MaskedColumn  6232
          mass_min float64 MaskedColumn  6232
          mass_url   str64       Column     0
             msini float64 MaskedColumn  6720
         msini_max float64 MaskedColumn  6744
         msini_min float64 MaskedColumn  6744
         msini_url   str64       Column     0
          bestmass float64 MaskedColumn  5676
      bestmass_min float64 MaskedColumn  5675
      bestmass_max float64 MaskedColumn  5675
      bestmass_url   str64       Column     0
         mass_prov    str5       Column     0
              

In [13]:
[sim_objects]=load(['sim_objects'])
sim_objects.info

<Table length=3359>
  name   dtype                   description                  
------- ------- ----------------------------------------------
main_id   str31                  Main identifier for an object
    ids str1082 List of all identifiers concatenated with pipe
   type    str4                                               

In [59]:
if exomercat['i_url'].dtype == np.dtype('<U64'):
    exomercat['i_url']=exomercat['i_url'].astype(object)
print(exomercat.info)

<Table length=7827>
       name         dtype     class     n_bad
------------------ ------- ------------ -----
                id   int64       Column     0
              name   str31       Column     0
              host   str26       Column     0
            letter    str1       Column     0
            ra_off float64       Column     0
           dec_off float64       Column     0
              mass float64 MaskedColumn  6148
          mass_max float64 MaskedColumn  6232
          mass_min float64 MaskedColumn  6232
          mass_url   str64       Column     0
             msini float64 MaskedColumn  6720
         msini_max float64 MaskedColumn  6744
         msini_min float64 MaskedColumn  6744
         msini_url   str64       Column     0
          bestmass float64 MaskedColumn  5676
      bestmass_min float64 MaskedColumn  5675
      bestmass_max float64 MaskedColumn  5675
      bestmass_url   str64       Column     0
         mass_prov    str5       Column     0
              

In [45]:
exomercat.info
#str26, str1,...

<Table length=7827>
       name         dtype     class     n_bad
------------------ ------- ------------ -----
                id   int64       Column     0
              name   str31       Column     0
              host   str26       Column     0
            letter    str1       Column     0
            ra_off float64       Column     0
           dec_off float64       Column     0
              mass float64 MaskedColumn  6148
          mass_max float64 MaskedColumn  6232
          mass_min float64 MaskedColumn  6232
          mass_url   str64       Column     0
             msini float64 MaskedColumn  6720
         msini_max float64 MaskedColumn  6744
         msini_min float64 MaskedColumn  6744
         msini_url   str64       Column     0
          bestmass float64 MaskedColumn  5676
      bestmass_min float64 MaskedColumn  5675
      bestmass_max float64 MaskedColumn  5675
      bestmass_url   str64       Column     0
         mass_prov    str5       Column     0
              

In [9]:
print(exomercat.colnames)
print(exomercat['bestmass_url'])

['id', 'name', 'host', 'letter', 'ra_off', 'dec_off', 'mass', 'mass_max', 'mass_min', 'mass_url', 'msini', 'msini_max', 'msini_min', 'msini_url', 'bestmass', 'bestmass_min', 'bestmass_max', 'bestmass_url', 'mass_prov', 'p', 'p_max', 'p_min', 'p_url', 'r', 'r_max', 'r_min', 'r_url', 'a', 'a_max', 'a_min', 'a_url', 'e', 'e_max', 'e_min', 'e_url', 'i', 'i_max', 'i_min', 'i_url', 'main_id', 'binary', 'discovery_method', 'status', 'status_string', 'confirmed', 'yod', 'alias', 'catalog', 'update_time', 'mismatch_flag_host']
                         bestmass_url                         
--------------------------------------------------------------
https://ui.adsabs.harvard.edu/abs/2019AJ....157..149L/abstract
https://ui.adsabs.harvard.edu/abs/2017AJ....153..136S/abstract
https://ui.adsabs.harvard.edu/abs/2008ApJ...672..553L/abstract
                                                           OEC
                                                            EU
https://ui.adsabs.harvard.edu/abs/2

In [16]:
exomercat['bestmass_max'].fill_value=999
exomercat['bestmass_min'].fill_value=-999
exomercat['bestmass_max']=exomercat['bestmass_max'].filled()
exomercat['bestmass_min']=exomercat['bestmass_min'].filled()

exomercat['bestmass_err']=np.maximum(exomercat['bestmass_max'],-exomercat['bestmass_min'])

<class 'astropy.table.column.Column'>
0.077 -0.077


0.077
0.16
1.5
0.245
0.57
0.15
0.05
999.0
0.02
...
8.5


In [682]:
exomercat['p_name']=exomercat['name']
exomercat['host_main_id']=exomercat['main_id']

for i in range(len(exomercat)):
    if type(exomercat['main_id'][i])!=np.ma.core.MaskedConstant:
        hostname=exomercat['main_id'][i]
    else:
        hostname=exomercat['host'][i]
    if type(exomercat['binary'][i])!=np.ma.core.MaskedConstant:
        exomercat['host_main_id'][i]=hostname+' '+exomercat['binary'][i]
    else:
        exomercat['host_main_id'][i]=hostname
    exomercat['p_name'][i]=exomercat['host_main_id'][i]+' '+exomercat['letter'][i]
print(exomercat['p_name','name'])
#ok so I can reconstruct host simbad main id and planet simbad main id. it needs a bit more writing using mask.nonzero()[0] and inverse of it
#why do I need them? because in ident I need the main id and all the others in id
#damn, there are some stars with no main id

#do I have distance cut opption? no, so I need main_id to get distance cut

/tmp/ipykernel_105243/2099565807.py:13: StringTruncateWarning: truncated right side string(s) longer than 31 character(s) during assignment
  exomercat['p_name'][i]=exomercat['host_main_id'][i]+' '+exomercat['letter'][i]
/tmp/ipykernel_105243/2099565807.py:13: StringTruncateWarning: truncated right side string(s) longer than 31 character(s) during assignment
  exomercat['p_name'][i]=exomercat['host_main_id'][i]+' '+exomercat['letter'][i]
/tmp/ipykernel_105243/2099565807.py:13: StringTruncateWarning: truncated right side string(s) longer than 31 character(s) during assignment
  exomercat['p_name'][i]=exomercat['host_main_id'][i]+' '+exomercat['letter'][i]
/tmp/ipykernel_105243/2099565807.py:13: StringTruncateWarning: truncated right side string(s) longer than 31 character(s) during assignment
  exomercat['p_name'][i]=exomercat['host_main_id'][i]+' '+exomercat['letter'][i]
/tmp/ipykernel_105243/2099565807.py:13: StringTruncateWarning: truncated right side string(s) longer than 31 charact

            p_name                   name       
----------------------------- ------------------
                  *   6 Lyn b            6 Lyn b
                  *   8 UMi b            8 UMi b
                  *  11 Com b           11 Com b
                  *  11 UMi b           11 UMi b
                  *  14 And b           14 And b
                  *  14 Her b           14 Her b
                          ...                ...
               [LPS2002]  9 b        M51-ULS-1 b
           [NC98] Cha HA  8 b         Cha Ha 8 b
[OKV2010] TYC 2627-638-1B A b       TYC 2627 A b
 [PGZ2001] J161031.9-191305 b    USco1610-1913 b
              [PLV2002] CX1 b      NGC 6440 X2 b
  [SLS2012] PYC J22502+2325 b 2MASS J2250+2325 b
     [TLM2010] 2M J044145 B b     2MASS 044144 b
Length = 7827 rows


In [673]:
for i in temp['main_id'].mask.nonzero()[0]:
    print(exomercat['name','host'][i])
#so those I won't be able to include as I don't know their distance limit...

       name         host  
----------------- --------
CoRoT-102719528 b HAT-P-58
  name    host 
-------- ------
K2-316 c K2-316
   name      host  
--------- ---------
LSR J1835 LSR J1835
      name             host      
---------------- ----------------
CWISE J2035-4936 CWISE J2035-4936
         name                  host        
--------------------- ---------------------
USco J155150.2-213457 USco J155150.2-213457


In [526]:
for cat in [cats[n],sim_new[n],sources]:
    print(cat.colnames)
    print(cat)
    
#wait, why do I have none in the last one? why does ident look like h_link?? because I forgot that arrays start at 0
#many h_link parent object_idref with 999999 with must be null value... 
#does this happen when h_link initialized and then directly ident then h_link too? yes. ok so need to change from join type left to normal
#then need to make sure either ident always run before h_link or that we don't loose objects eithe way.

#why do I have only 800 objects ?

['ref', 'provider_name', 'provider_url', 'provider_bibcode', 'source_id']
        ref         provider_name ...   provider_bibcode  source_id
------------------- ------------- ... ------------------- ---------
                           SIMBAD ... 2000A&AS..143....9W         1
1980AJ.....85..454H        SIMBAD ... 2000A&AS..143....9W         2
1986MNRAS.223..629M        SIMBAD ... 2000A&AS..143....9W         3
1988AJ.....95.1841G        SIMBAD ... 2000A&AS..143....9W         4
1988IRASP.C......0J        SIMBAD ... 2000A&AS..143....9W         5
1991AJ....101.2220B        SIMBAD ... 2000A&AS..143....9W         6
                ...           ... ...                 ...       ...
2019AJ....158..174D        SIMBAD ... 2000A&AS..143....9W       164
2019AJ....158..182G        SIMBAD ... 2000A&AS..143....9W       165
2020A&A...641A..23P        SIMBAD ... 2000A&AS..143....9W       166
2020A&A...642A.179M        SIMBAD ... 2000A&AS..143....9W       167
2020AJ....160...83S        SIMBAD ... 2000

In [457]:
print(sim_h_link)
#huch didn't I exclude moving groups? Ah I believe is what happens when I create object_idref from it

       main_id                parent_main_id        ... membership
                                                    ...  percent  
--------------------- ----------------------------- ... ----------
              G 236-1      NAME Hyades Moving Group ...        100
             G 190-27    NAME Pleiades Moving Group ...         75
      SCR J0630-7643B      NAME AB Dor Moving Group ...         75
            HD  82443    NAME Pleiades Moving Group ...        100
       PM J20502-3424 NAME Carina-near Moving Group ...         75
       PM J20300+0023        NAME Argus Association ...         54
                  ...                           ... ...        ...
            V* AB Dor    NAME Pleiades Moving Group ...        100
1RXS J211605.3+295156              NAME Octans-near ...         60
       PM J10367+1521  NAME Ursa Major Moving Group ...         75
       PM J10367+1521  NAME Ursa Major Moving Group ...         75
            HD 230017      NAME Castor Moving Group ...       

In [261]:
#wanted to create table from column so that I can use ap.table.join with single column
t=ap.table.Table()
t['a']=[1,2]
print(t)
s=ap.table.Table(t['a'])
print(s)
print(type(t),type(t['a']))
print(type(t['a'].to_table()))

 a 
---
  1
  2
col0 col1
---- ----
   1    2
<class 'astropy.table.table.Table'> <class 'astropy.table.column.Column'>


AttributeError: 'Column' object has no attribute 'to_table'

In [516]:
def update_database_tables(new,table):
    """
    preferably do sources befor any other tables
    :param update: list of astropy tables corresponding
    """
    [sources,objects,ident,h_link,star_basic,planet_basic,disk_basic,mesDist]=load(['sources','objects','ident',
                                    'h_link','star_basic','planet_basic','disk_basic','mesDist'])
    
    if table=='h_link':
        h_link=ap.table.vstack([new,h_link])
        h_link=ap.table.unique(h_link,silent=True)
        h_link.remove_columns(['child_object_idref','parent_object_idref'])
        h_link=ap.table.join(h_link,objects['main_id','object_id'],join_type='left')
        h_link.rename_column('object_id','child_object_idref')
        h_link.remove_column('main_id')
        h_link=ap.table.join(h_link,objects['main_id','object_id'],
                             keys_left='parent_main_id',keys_right='main_id')
        h_link.rename_column('object_id','parent_object_idref')
        h_link.remove_columns(['main_id','parent_main_id'])
    if table=='mesDist':
        mesDist=ap.table.vstack([new,mesDist])
        mesDist=ap.table.unique(mesDist,silent=True)
        mesDist.remove_column('object_idref')
        mesDist=ap.table.join(mesDist,objects['main_id','object_id'],join_type='left')
        mesDist.rename_column('object_id','object_idref')
        mesDist.remove_column('main_id')
    if table=='planet_basic':
        planet_basic=ap.table.vstack([new,planet_basic])
        planet_basic=ap.table.unique(planet_basic,keys='main_id')
        planet_basic.remove_column('object_idref')
        planet_basic=ap.table.join(planet_basic,objects['main_id','object_id'],join_type='left')
        planet_basic.rename_column('object_id','object_idref')
        planet_basic.remove_column('main_id')
    if table=='star_basic':
        star_basic=ap.table.vstack([new,star_basic])
        star_basic=ap.table.unique(star_basic,silent=True)
        star_basic.remove_column('object_idref')
        star_basic=ap.table.join(star_basic,objects['main_id','object_id'],join_type='left')
        star_basic.rename_column('object_id','object_idref')
        star_basic.remove_column('main_id')
    
    if table=='ident':
        ident=ap.table.vstack([new,ident])
        ident=ap.table.unique(ident,silent=True)
        ident.remove_column('object_idref')
        ident=ap.table.join(ident,objects['main_id','object_id'],join_type='left')
        ident.rename_column('object_id','object_idref')
        ident.remove_column('main_id')
    if table=='objects':
        #add main id to other tables
        
        #then update object
        objects=ap.table.vstack([new,objects])
        objects=ap.table.unique(objects,silent=True)
        objects['object_id']=[j+1 for j in range(len(objects))]
        objects=objects['object_id','type','ids','main_id']#changing order of columns
        #then update object_idref
        #start with ident. need to update object_idref now
        #---------now outer join object_idref sorted by type to _basic to show missing data
        #planet_basic.remove_column('object_idref')
        #planet_basic=ap.table.join(planet_basic,objects['main_id','object_id'],join_type='outer')
        #planet_basic.rename_column('object_id','object_idref')
    
    def match(cat,sources,paras,idref=True):
        """
        This function joins sources and cat. If idref is True then it joins the column 
        ref from sources as f'{para}_ref' to cat. If idref is False it instead joins 
        source_id as source_idref to cat.
        """
        for para in paras:
            if idref:
                if len(cat)!=0 and len(sources)!=0:
                    if type(cat[para+'_source_idref'])==ap.table.column.MaskedColumn:
                        cat[para+'_source_idref'].fill_value=0
                        cat=cat.filled()
                    cat=ap.table.join(cat,sources['ref','source_id'],
                            keys_left=para+'_source_idref',keys_right='source_id',join_type='left')
                    cat.rename_column('ref',f'{para}_ref')
                    cat.remove_column('source_id')
            else:
                if para+'_ref' in cat.colnames:
                    if type(cat[para+'_ref'])==ap.table.column.MaskedColumn:
                        cat[para+'_ref'].fill_value=''
                        cat=cat.filled()
                    cat.remove_column(f'{para}_source_idref')
                    cat=ap.table.join(cat,sources['ref','source_id'],keys_left=para+'_ref',keys_right='ref',join_type='left')
                    cat.rename_column('source_id',f'{para}_source_idref')
                    cat.remove_columns(['ref',para+'_ref'])
                    #in case the para_value entry is masked this if environment will put the source_id entry to null
                    if para+'_value' in cat.colnames:
                        if type(cat[para+'_value'])==ap.table.column.MaskedColumn:
                            for i in cat[para+'_value'].mask.nonzero()[0]:
                                cat[f'{para}_source_idref'][i]=0
        return cat
    
    
    if table=='sources':
    #----------------check for duplicates--------
        
        #add refs to other tables
        
        #ident=match(ident,sources,['id'])
        #h_link=match(h_link,sources,['h_link'])
        #star_basic=match(star_basic,sources,['coo','plx','dist'])
        #planet_basic=match(planet_basic,sources,['mass'])
        #disk_basic=match(disk_basic,sources,['rad'])
        #mesDist=match(mesDist,sources,['dist'])
        n=len(sources)
        sources=ap.table.join(sources,new,join_type='left') 
        #sources=ap.table.vstack([new,sources])
    #ref in sources unique
        #sources=ap.table.unique(sources,silent=True)
    #main_id unique
    #--------------add data----------------
    #-----------------update foreign keys-------------
    ###-------------------update source id-------------
        #sources['source_id'].mask.nonzero()[0] 
        #[j+1 for j in range(n,len(sources))]
        sources['source_id']=[j+1 for j in range(len(sources))]#introduces an identifier column

    ###-------------------update source id ref------------
    #this update is problematic as ref is not nessecarely unique
        
    if len(sources)==0:
        print('please first ingest some sources data')
    else:
        ident=match(ident,sources,['id'],False)
        h_link=match(h_link,sources,['h_link'],False)
        star_basic=match(star_basic,sources,['coo','plx','dist'],False)
        planet_basic=match(planet_basic,sources,['mass'],False)
        disk_basic=match(disk_basic,sources,['rad'],False)
        mesDist=match(mesDist,sources,['dist'],False)
    
    
    ###-------------------update object id(ref)----------------
    #I can easily write this in a for loop though that would take longer than np.where assignment
    #np.where assignment issue
    

    #stars['object_idref']=['S%04d' % (j+1) for j in range(len(stars))]

    ###-------------------update parent-----------------
    
    save([sources,objects,ident,h_link,star_basic,planet_basic,disk_basic,mesDist],
         ['sources','objects','ident','h_link','star_basic','planet_basic','disk_basic','mesDist'])
    return sources,objects,ident,h_link,star_basic,planet_basic,disk_basic,mesDist

# id_source_idref assigning issue
ok now I tried a lot around to get ident source_idref assigned. if it works try backtracking and getting code easier again

either I need to explicitely name each table and have rest in function
or I look up again how alias and so on works. only found with or without [:] in end stands for copy false or true

I can solve this issue later, way around is to save and load tables
...still not working, will look at this later

In [192]:
print(sources)#why only 60, was the rest double? can't be, there is an unique function in the sources function
#ah because only ident sources added to sources... does not work, would just be one. ah saving got forgotten
print(sim_sources)
print(ident)#source id not assigned. 
print(sim_ident)

#well at least some are assigned

        ref         provider_name ...   provider_bibcode  source_id
------------------- ------------- ... ------------------- ---------
                           SIMBAD ... 2000A&AS..143....9W         1
1980AJ.....85..454H        SIMBAD ... 2000A&AS..143....9W         2
1986MNRAS.223..629M        SIMBAD ... 2000A&AS..143....9W         3
1988AJ.....95.1841G        SIMBAD ... 2000A&AS..143....9W         4
1988IRASP.C......0J        SIMBAD ... 2000A&AS..143....9W         5
1991AJ....101.2220B        SIMBAD ... 2000A&AS..143....9W         6
                ...           ... ...                 ...       ...
2019AJ....158..174D        SIMBAD ... 2000A&AS..143....9W       164
2019AJ....158..182G        SIMBAD ... 2000A&AS..143....9W       165
2020A&A...641A..23P        SIMBAD ... 2000A&AS..143....9W       166
2020A&A...642A.179M        SIMBAD ... 2000A&AS..143....9W       167
2020AJ....160...83S        SIMBAD ... 2000A&AS..143....9W       168
2020yCat.1350....0G        SIMBAD ... 2000A&AS..

# old stuff

In [39]:
def query(link,query,catalogs=[]):
    """Performs a query via TAP on the service given in the link parameter. 
    If catalogs is specified those tables are uploaded to the service. 
    :param link: service URL
    :param query: query in ADQL 
    :param catalogs: list of tables to be uploaded to the service
    :return result.to_table(): astropy table containing the result of the query"""
    service = vo.dal.TAPService(link)
    if catalogs==[]:
        result=service.run_async(query.format(**locals()), maxrec=160000)
    else:
        tables={}
        for i in range(len(catalogs)):
            tables.update({f"t{i+1}":catalogs[i]})
        result = service.run_async(query,uploads=tables,timeout=None, maxrec=160000)
    return result.to_table()

def save(cat,path):
    """
    Saves cat in the location path as .xml file.
    :param cat: astropy table to be saved
    :param path: path to where to save the cat table
    """
    ap.io.votable.writeto(ap.io.votable.from_table(cat), f'{path}.xml')
    return


def sources_table(cat,ref_columns,provider,old_sources=ap.table.Table()): #put this into source function
    """
    """
    cat_sources=ap.table.Table()
    cat_reflist=[]
    for k in range(len(ref_columns)):
        if type(cat[ref_columns[k]])==ap.table.column.MaskedColumn:
            cat_reflist.extend(cat[ref_columns[k]][np.where(cat[ref_columns[k]].mask==False)])
        else:
            cat_reflist.extend(cat[ref_columns[k]])
    cat_sources['ref']=cat_reflist
    cat_sources=ap.table.unique(cat_sources)#keeps only unique values
    #attaches service information
    cat_sources['provider_name']=[provider[0]]*len(cat_sources)
    cat_sources['provider_url']=[provider[1]]*len(cat_sources)
    cat_sources['provider_bibcode']=[provider[2]]*len(cat_sources)
    sources=ap.table.vstack([old_sources,cat_sources])
    sources=ap.table.unique(sources)
    return sources

def match_ref_to_source_id(cat,para_ref,para,para_value,provider_name=''):
    """
    This function matches the reference in the para_ref column of the cat table to the 
    ref column in the sources table and adds the corresponding source_id in the for
    that created column para_source_idref of cat.
    """
    #the following line is needed as without it I get the TableMergeError: 
    #Columns have incompatible types ['object', 'str1984']
    cat[para_ref]=cat[para_ref].astype(str)#even though type is already string without this I get type conversion issues
    #the following if statement makes sure there are no masked entries in cat[para_ref] as the join would not work otherwise
    if type(cat[para_ref])==ap.table.column.MaskedColumn:
        cat[para_ref].fill_value=''
        cat=cat.filled()
    #here the join is performed with the option to only have part of the source table joined. That is useful for the
    #special case in which there is a value in para_value but no reference in para_ref. In this case the source_id
    #will point to the provider only.
    if provider_name=='':
        joined_sources=sources['ref','source_id']
    else:
        joined_sources=sources['ref','source_id'][np.where(sources['provider_name']==provider_name)]
    cat=ap.table.join(cat,joined_sources,keys_left=para_ref,keys_right='ref',join_type='left')
    cat.rename_column('source_id',f'{para}_source_idref')
    #next line removes the column which would otherwise just be a duplicate of cat[para_ref]
    cat.remove_column('ref')
    #in case the para_value entry is masked this if environment will put the source_id entry to null
    if type(cat[para_value])==ap.table.column.MaskedColumn:
        for i in cat[para_value].mask.nonzero()[0]:
            cat[f'{para}_source_idref'][i]=0
    return cat



#defining queries
select="""SELECT b.main_id,b.ra,b.dec,b.oid, b.coo_bibcode, 
b.plx_err, b.plx_value, b.plx_bibcode, h_link.membership , 
h_link.parent, h_link.link_bibcode, a.otypes,ids.ids
"""
adql_query=[select+
"""
FROM basic AS b
JOIN ids ON b.oid=ids.oidref
    JOIN alltypes AS a ON b.oid=a.oidref
        LEFT JOIN h_link ON b.oid=h_link.child
WHERE b.plx_value >=50.
""",
"""
SELECT *
FROM exomercat.exomercat
"""]
upload_query=[select+"""
FROM basic AS b
JOIN ids ON b.oid=ids.oidref
    JOIN alltypes AS a ON b.oid=a.oidref
        LEFT JOIN h_link ON b.oid=h_link.child
            JOIN TAP_UPLOAD.t1 ON b.oid=t1.parent
WHERE (b.plx_value IS NULL) AND (otype='**..')""",
select+"""
FROM basic AS b
JOIN ids ON b.oid=ids.oidref
    JOIN alltypes AS a ON b.oid=a.oidref
        LEFT JOIN h_link ON b.oid=h_link.child
            JOIN TAP_UPLOAD.t1 ON b.oid=t1.oid
WHERE (b.plx_value IS NULL) AND (otype='Pl..')
""",
"""SELECT *
FROM mesDistance
JOIN TAP_UPLOAD.t1 ON oidref=t1.oid

"""]


In [366]:
def disks_table_creation():
    """
    This function reads in the disk data provided by 
    Grant Kennedy. Then it adds the columns for 
    object identifiers, object type and source reference. 
    It proceeds by matching the references to the ones 
    in the source table resulting in another column for 
    the source identifier. Finally the new disks data 
    table is saved.
    """
    sources=ap.io.votable.parse_single_table(
        "data/sources.xml").to_table()
    #reads in the disk data provided by Grant Kennedy
    disks=ap.io.votable.parse_single_table(
        "data/Grant_absil_2013_.xml").to_table()
    #adds the columns for object identifiers, object type 
    #and source reference
    disks['object_idref']=['D%04d' % (j+1) 
                           for j in range(len(disks))]
    disks['type']=['di' for j in range(len(disks))]
    disks['disks_ref']=['Grant Kennedy, priv. comm.' 
                        for j in range(len(disks))]
    #matching the references to the ones in the source 
    #table resulting in another column for the source identifier
    disks=match_ref_to_source_id(disks[:],'disks_ref',
                                 'disks','rdisk_bb','None')
    #new disks data table is saved
    ap.io.votable.writeto(ap.io.votable.from_table(disks),
                          f'data/disks.xml')
    return

def query_simbad():
    """
    This function queries from the SIMBAD database all the 
    objects within 20pc of the Sun. It obtains for each the entries
    for the parameters main identifier, right ascencion, declination,
    object internal identifier, bibcode of the coordinate parameter, 
    parallax value, parallax error, parallax bibcode, object internal
    identifier of the parent object, likelyhood of the parent relation,
    bibcode of the parent relation, object types and identifiers. It 
    then sorts the objects into star, system and planet types. Finally
    the table is saved.
    """

    #queries SIMBAD for objects with parallax value > 50 mas
    TAP_service="http://simbad.u-strasbg.fr:80/simbad/sim-tap"
    simbad=query(TAP_service,adql_query[0])
    #adds parent and children objects with no parallax value
    parents_without_plx=query(TAP_service,upload_query[0],[simbad])
    children_without_plx=query(TAP_service,upload_query[1],[simbad])
    #adding of no_parallax objects to rest of simbad query objects
    simbad=ap.table.vstack([simbad,parents_without_plx])
    simbad=ap.table.vstack([simbad,children_without_plx])
    #sorting from object type into star, system and planet type
    simbad['type']=['None' for i in range(len(simbad))]
    simbad['multiple']=[False for i in range(len(simbad))]
    for i in range(len(simbad)):
        if "Pl" in simbad['otypes'][i]:
            simbad['type'][i]='pl'
        elif "*" in simbad['otypes'][i]:
            if "**" in simbad['otypes'][i]:
                simbad['type'][i]='sy'
                simbad['multiple'][i]=True
            else:
                simbad['type'][i]='st'
        else:
            print('Neither Pl,* or **:',simbad['otypes'][i])
    #saving table
    ap.io.votable.writeto(ap.io.votable.from_table(simbad),
                          f'data/simbad.xml')
    return


def stars_table_creation():
    """
    This function reads in the simbad, sources and best_paras data 
    obtained through the similarly named functions. Then it
    removes the planet objects from the simbad table, adds the 
    column for object identifiers and proceeds by matching the 
    parameter source references to the ones in the source table 
    resulting in another column of source identifiers for each. 
    It then joins the best_paras table for the stellar multiple 
    measurement parameters. Finally the stars data table is saved.
    """
    simbad=ap.io.votable.parse_single_table("data/simbad.xml").to_table()
    sources=ap.io.votable.parse_single_table("data/sources.xml").to_table()
    best_paras=ap.io.votable.parse_single_table("data/best_paras.xml").to_table()
    #simbad table without planet type objects
    temp_stars=simbad[np.where(simbad['type']!='pl')]
    #removing double objects
    stars=ap.table.Table(ap.table.unique(temp_stars,keys='oid'),copy=True)
    #assigning object identifiers
    stars['object_idref']=['S%04d' % (j+1) for j in range(len(stars))]
    #matching source freferences to source identifier
    stars=match_ref_to_source_id(stars[:],'coo_bibcode','coord','ra','SIMBAD')
    stars=match_ref_to_source_id(stars,'plx_bibcode','plx','plx_value','SIMBAD')
    ap.io.votable.writeto(ap.io.votable.from_table(stars),
                          f'data/stars_wt_bestpara.xml')
    #add best para from multiple measurements tables
    stars=ap.table.join(stars,best_paras,keys='object_idref',join_type='left')
    #saving table
    ap.io.votable.writeto(ap.io.votable.from_table(stars),
                          f'data/stars.xml')
    return

#I might want to not use exomercat but instead some of the databases it is based on.
def query_exomercat(temp=False):
    """
    This function queries the Exo-MerCat catalogue and returns 
    everything in it. It then saves the data in a table.
    """
    stars=ap.io.votable.parse_single_table("data/stars.xml").to_table()
    TAP_sercive="http://archives.ia2.inaf.it/vo/tap/projects"
    if temp:
        exomercat=ap.io.votable.parse_single_table("data/raw_exomercat.xml").to_table()
        #exomercat['type']=['pl' for j in range(len(exomercat))]
        #exomercat['object_idref']=['P%04d' % (j+1) for j in range(len(exomercat))]
    else:
        exomercat=query(TAP_service,adql_query[1])
    ap.io.votable.writeto(ap.io.votable.from_table(exomercat),
                          f'data/exomercat.xml')
    return

def sources_table_creation():
    """
    This function creates the provenance information by first reading
    in the stars, exomercat, simbad, disks and mesDist data obtained
    through the similarly named functions. The h_link table gets defined
    as a subset of the simbad one by removing rows with parent relation
    probability smaller than 50%. From all those tables the reference 
    column entries are stored in the source table. Columns for provider
    name, provider bibcode and provider url are added together with
    a source identifier. In case of provider servers a row for data without
    producer reference is added. Finally the sources data table is saved.
    """
    stars=ap.io.votable.parse_single_table("data/stars.xml").to_table()
    exomercat=ap.io.votable.parse_single_table("data/exomercat.xml").to_table()
    simbad=ap.io.votable.parse_single_table("data/simbad.xml").to_table()
    disks=ap.io.votable.parse_single_table("data/disks.xml").to_table()
    mesDist=ap.io.votable.parse_single_table("data/mesDist.xml").to_table()
    
    sources=ap.table.Table()
    h_link=simbad['oid','membership','parent','link_bibcode',
                  'otypes','multiple'][np.where(simbad['membership']>50)]
    
    TAP_service=["http://simbad.u-strasbg.fr:80/simbad/sim-tap",
            "http://archives.ia2.inaf.it/vo/tap/projects","None"]
    provider_name=['SIMBAD','Exo-MerCat','None']
    provider_bibcode=['2000A&AS..143....9W','2020A&C....3100370A','None']
    ref_columns=[#simbad queried reference columns
        [['coo_bibcode','plx_bibcode'],['link_bibcode'],['bibcode']],
                    [['bestmass_url']],#exomercat queried reference columns
                    [['disks_ref']]]#no provider reference columns
    tables=[[stars, h_link, mesDist],
            [exomercat],
            [disks]]
    for i in range(3):
        for cat,ref in zip(tables[i],ref_columns[i]):
            sources=sources_table(cat,ref,[provider_name[i],TAP_service[i],
                                           provider_bibcode[i]],sources)
        #add row for provider data without producer reference in case of simbad and exomercat
        if i != 2:
            sources.add_row(['',provider_name[i],TAP_service[i],provider_bibcode[i]])
    
    sources['source_id']=[j+1 for j in range(len(sources))]#introduces an identifier column
    ap.io.votable.writeto(ap.io.votable.from_table(sources),
                          f'data/sources.xml')
    return

def ident_table_creation(temp=False):
    """
    This function reads in the sources, stars, exomercat, disks and simbad data 
    obtained through the similarly named functions. It then initializes the ident
    table with the columns object_idref, id and id_source_idref. It 
    """
    sources=ap.io.votable.parse_single_table("data/sources.xml").to_table()
    stars=ap.io.votable.parse_single_table("data/stars.xml").to_table()
    exomercat=ap.io.votable.parse_single_table("data/exomercat.xml").to_table()
    disks=ap.io.votable.parse_single_table("data/disks.xml").to_table()
    simbad=ap.io.votable.parse_single_table("data/simbad.xml").to_table()
    exo_planets=ap.io.votable.parse_single_table("data/exo_planets.xml").to_table()
    sim_planets=ap.io.votable.parse_single_table("data/sim_planets.xml").to_table()

    
    
    #this I could rewrite as a join though I am not sure if that is worth it
    #temp=ap.table.Table(names=['ref','provider_name'],[['','SIMBAD'],['','Exo-MerCat']]
    #exomercat_source_idref=ap.table.join(sources,temp)['source_id'][np.where()]
    noProducer=sources[np.where(sources['ref']=='')]
    for i in range(len(noProducer)):
        if noProducer['provider_name'][i]=='SIMBAD':
            sim_source_idref=noProducer['source_id'][i]
        if noProducer['provider_name'][i]=='Exo-MerCat':
            exmercat_source_idref=noProducer['source_id'][i]
    disk_source_idref=sources['source_id'][np.where(sources['ref']=='Grant Kennedy, priv. comm.')][0]
            
    if temp:
        ident=ap.table.Table(names=['object_idref','id','id_source_idref'],
                    dtype=['str_','str','int64'])
        exomercat['type']=['pl' for j in range(len(exomercat))]
        exomercat['object_idref']=['P%04d' % (j+1) for j in range(len(exomercat))]
        exomercat['ids']=exomercat['name']
        cat=exomercat #fakeplanets if you want to test match found while exomercat only limited sample available
        #this is currently not needed as only one entry in exomercat['name'], 
        #but if I change it I get again issue with source table creation
        #wait if I read all sim_planets ids into ident I can join exomercat on leftkey id, rightkey 'name'
        for i_exo in range(len(cat)):
            for ID in cat['ids'][i_exo].split('|'):
                ident.add_row([cat['object_idref'][i_exo],ID,exmercat_source_idref])

        sim_planets['object_idref']=['P0000' for j in range(len(sim_planets))]
        n=len(exomercat)+1
        #this might be good if I want to add later on but now that I only have one element in exomercat['name']
        #it might be possible to simplify it
        for i_sim in range(len(sim_planets)):
            for ID in sim_planets['ids'][i_sim].split('|'):
                if ID in ident['id']:
                    sim_planets['object_idref'][i_sim]=ident['object_idref'][np.where(ident['id']==ID)][0]
                    ident.add_row([sim_planets['object_idref'][i_sim],ID,sim_planets_source_idref])
            if sim_planets['object_idref'][i_sim]=='P0000':
                sim_planets['object_idref'][i_sim]='P%04d' % n
                n+=1
                for ID in sim_planets['ids'][i_sim].split('|'):
                    ident.add_row([sim_planets['object_idref'][i_sim],ID,sim_planets_source_idref])
        print('before stars added',ident[np.where(ident['id']=='UCAC3 166-137374')])
        for i_stars in range(len(stars)):
            for ID in stars['ids'][i_stars].split('|'):
                ident.add_row([stars['object_idref'][i_stars],ID,sim_planets_source_idref])
        print('after stars added',ident[np.where(ident['id']=='UCAC3 166-137374')])
        disks['disk_source_idref']=[disk_source_idref for j in range(len(disks))]## is this really needed?
        temp_disks=ap.table.Table(disks['object_idref','id','disk_source_idref'],copy=True)
        temp_disks.rename_column('disk_source_idref','id_source_idref')
        ident=ap.table.vstack([ident,temp_disks])#this is not correct yet
        #so how to get source_idref and id_source_idref the same?
    else:
        print('doing stars...')
        #add stars to ident
        #for i_stars in range(len(stars)):
         #   for ID in stars['ids'][i_stars].split('|'):
          #      ident.add_row([stars['object_idref'][i_stars],ID,sim_planets_source_idref])
        ident_query="""SELECT id, t1.*
        FROM ident 
        JOIN TAP_UPLOAD.t1 ON oidref=t1.oid
            """
        ident=query("http://simbad.u-strasbg.fr:80/simbad/sim-tap",ident_query,[stars['oid','object_idref'][:].copy()])
        ident['id_source_idref']=[sim_source_idref for j in range(len(ident))]
        if len(exo_planets) !=0:
            exo_planets['id_source_idref']=[exmercat_source_idref for j in range(len(exo_planets))]
            #add to ident
            ident['object_idref']=ident['object_idref'].astype(str)
            ident=ap.table.vstack([ident,exo_planets['object_idref','id','id_source_idref']]) 
        print('everything working till here')
        #check if sim_planets already present as exo_planets and add to ident
        sim_planets['object_idref']=['P0000' for j in range(len(sim_planets))]
        n=len(exo_planets)+1
        #this might be good if I want to add later on but now that I only have one element in exomercat['name']
        #it might be possible to simplify it
        print('wait I can improve this too by querying main_id in exo_planets and matching on that')
        for i_sim in range(len(sim_planets)):
            for ID in sim_planets['ids'][i_sim].split('|'):
                if ID in ident['id']:
                    sim_planets['object_idref'][i_sim]=ident['object_idref'][np.where(ident['id']==ID)][0]
                    ident.add_row([sim_planets['object_idref'][i_sim],ID,sim_planets_source_idref])
            if sim_planets['object_idref'][i_sim]=='P0000':
                sim_planets['object_idref'][i_sim]='P%04d' % n
                n+=1
                for ID in sim_planets['ids'][i_sim].split('|'):
                    ident.add_row([sim_planets['object_idref'][i_sim],ID,sim_planets_source_idref])
        #add disks to ident
        print('doing disks...')
        disks['disk_source_idref']=[disk_source_idref for j in range(len(disks))]## is this really needed?
        temp_disks=ap.table.Table(disks['object_idref','id','disk_source_idref'],copy=True)
        temp_disks.rename_column('disk_source_idref','id_source_idref')
        ident=ap.table.vstack([ident,temp_disks])#this is not correct yet
        save(exo_planets,'data/exo_planets')
    save(sim_planets,'data/sim_planets')
    save(ident,'data/ident')
    return

def planets_table_creation(temp=False):
    sources=ap.io.votable.parse_single_table("data/sources.xml").to_table()
    ident=ap.io.votable.parse_single_table("data/ident.xml").to_table()
    exomercat=ap.io.votable.parse_single_table("data/exomercat.xml").to_table()
    simbad=ap.io.votable.parse_single_table("data/simbad.xml").to_table()


    #either I create additional row with only one digit format and compare to 'P' with np.where
    #or I include type in ident and compare to that
    #copying sounds easier
    if temp: 
        exo_planets=ap.io.votable.parse_single_table("data/exo_planets.xml").to_table()
        sim_planets=ap.io.votable.parse_single_table("data/sim_planets.xml").to_table()
        group=ident[np.where(np.array(ident['object_idref'],dtype='S1')==b'P')]
        group=group.group_by('object_idref')
        planets=ap.table.Table(names=['object_idref','ids'],
                    dtype=['str_','str'])
        ind=group.groups.indices
        for i in range(len(ind)-1):
            ids="|".join(group['id'][ind[i]:ind[i+1]])
            planets.add_row([group['object_idref'][ind[i]],ids])
        planets=ap.table.join(planets,exomercat['object_idref','bestmass','bestmass_url','host'],keys='object_idref',join_type='left')
        planets=ap.table.join(planets,sim_planets['oid','object_idref'],keys='object_idref',join_type='left')
        #sort out all exomercat objects not within our distance cut
        #now join sim_planets
        #now get object_idref
    else:
        #creating planets from simbad '
        temp_sim_planets=simbad['oid','ids','type','membership','parent','link_bibcode'][np.where(simbad['type']=='pl')]
        sim_planets=ap.table.Table(ap.table.unique(temp_sim_planets,keys='oid'),copy=True)
        ap.io.votable.writeto(ap.io.votable.from_table(sim_planets),
                          f'data/sim_planets.xml')
        
        #creating planets from exomercat
        host_id_query="""SELECT main_id as host_main_id, oid as parent_oid,t1.*
            FROM basic
            JOIN ident ON oid = oidref
            JOIN TAP_UPLOAD.t1 ON ident.id=t1.host
            """
        exo_planets=query("http://simbad.u-strasbg.fr:80/simbad/sim-tap",host_id_query,[exomercat[:].copy()])
        exo_planets=ap.table.join(exo_planets,stars['object_idref','main_id'],keys_left='host_main_id',keys_right='main_id')
        exo_planets.remove_columns(['main_id','id'])
        exo_planets.rename_columns(['object_idref','name'],['parent_object_idref','id'])
        planet_id_query="""SELECT main_id, oid ,t1.*
            FROM basic
            JOIN ident ON oid = oidref
            JOIN TAP_UPLOAD.t1 ON ident.id=t1.id
            """
        temp=query("http://simbad.u-strasbg.fr:80/simbad/sim-tap",planet_id_query,[exo_planets[:].copy()])
        exo_planets=ap.table.join(exo_planets,temp['main_id','oid','id'],keys='id',join_type='left')
        exo_planets['object_idref']=['P%04d' % (j+1) for j in range(len(exo_planets))]
        ap.io.votable.writeto(ap.io.votable.from_table(exo_planets),
                          f'data/exo_planets.xml')
        #joining both
        planets=ap.table.join(exo_planets,sim_planets,join_type='outer',keys='oid')
        print('tbd: clean up by deleting unneeded rows and puting what is missing in')
        #get object_idref, parent/parent_oid
        #in other functions: 
            #create clean ids
            #create id and id_source_idref
            #have parent_object_idref and h_link_source_idref
    planets['type']=['pl' for j in range(len(planets))]
    planets=match_ref_to_source_id(planets,'bestmass_url','bestmass','bestmass','Exo-MerCat')
    save(planets,'data/planets')
    return

def objects_table_creation():
    """
    This function reads in the stars, disks and planets data 
    obtained through the similarly named functions. Then it
    combines the planets, stars and disks data.
    """
    stars=ap.io.votable.parse_single_table("data/stars.xml").to_table()
    disks=ap.io.votable.parse_single_table("data/disks.xml").to_table()
    planets=ap.io.votable.parse_single_table("data/planets.xml").to_table()
    
    #the following line is needed as without it I get the TableMergeError: 
    #Columns have incompatible types ['object', 'str1984']
    stars['ids']=stars['ids'].astype(np.str_)
    
    objects=ap.table.vstack([planets['object_idref','oid','type','ids'],
                             stars['object_idref','oid','type','ids']])
    #disks ids missing
    objects=ap.table.vstack([objects,disks['object_idref','type']])
    objects.rename_column('object_idref', 'object_id')
    ap.io.votable.writeto(ap.io.votable.from_table(objects),
                          f'data/objects.xml')
    return

def h_link_table_creation():
    """
    This function reads in the sources, simbad, ident, objects and disks data 
    obtained through the similarly named functions.
    """
    sources=ap.io.votable.parse_single_table("data/sources.xml").to_table()
    simbad=ap.io.votable.parse_single_table("data/simbad.xml").to_table()
    h_link=simbad['oid','membership','parent','link_bibcode'][np.where(simbad['membership']>50)]
    ident=ap.io.votable.parse_single_table("data/ident.xml").to_table()
    objects=ap.io.votable.parse_single_table("data/objects.xml").to_table()
    disks=ap.io.votable.parse_single_table("data/disks.xml").to_table()
    #exo_planets=ap.io.votable.parse_single_table("data/exo_planets.xml").to_table() #once exomercat working again
    
    #simbad parent info
    temp=objects['object_id','oid'][np.where(objects['oid'].mask==False)]
    temp.rename_column('object_id','child_object_idref')

    h_link=ap.table.join(h_link,temp,keys='oid',join_type='left')
    #rename child_object_idref column in object into parent_object_idref
    #temp.rename_column('child_object_idref','parent_object_idref')
    #h_link.rename_column('oid','child_oid')
    #temp.rename_column('oid','parent_oid')
    #h_link.rename_column('parent','parent_oid')
    h_link.rename_columns(['oid','parent'],['child_oid','parent_oid'])
    temp.rename_columns(['child_object_idref','oid'],['parent_object_idref','parent_oid'])

    h_link=ap.table.join(h_link,temp,keys='parent_oid',join_type='left')
    h_link=h_link[np.where(h_link['parent_object_idref'].mask==False)]
    
    h_link.rename_column('link_bibcode','h_link_ref')
    h_link['h_link_ref']=h_link['h_link_ref'].astype(str)

    
    #exomercat parent info
    #exomercat['host'] join ident to get object_idref of parent
    #hm looks like I might be doing this stuff already in planets creation
    #exo_planets.rename_column('object_idref','child_object_idref')
    #hm ups, this already has id_source_idref... will this be an issue if idref function?
    #h_link=ap.table.vstack([h_link,exo_planets['child_object_idref','parent_object_idref','id_source_idref']])
    
    #disk parent info
    temp=ap.table.join(disks['object_idref','main_id','disks_ref'],ident['object_idref','id'],keys_left='main_id',keys_right='id')
    temp.rename_column('object_idref_1','child_object_idref')
    temp.rename_column('object_idref_2','parent_object_idref')
    temp.rename_column('disks_ref','h_link_ref')
    temp['membership']=[100 for j in range(len(temp))]
    temp.remove_column('main_id')
    temp.remove_column('id')
    h_link=ap.table.vstack([h_link,temp])
    
    h_link=match_ref_to_source_id(h_link[:],'h_link_ref','h_link','id')
    h_link.remove_column('parent_oid')
    h_link.remove_column('child_oid')
    h_link=ap.table.unique(h_link)#membership is masked because I have from disk, what value do I put here? 100%
    save(h_link,'data/h_link')
    return

def multimeas_table_creation():
    """
    
    """
    sources=ap.io.votable.parse_single_table("data/sources.xml").to_table()
    objects=ap.io.votable.parse_single_table("data/objects.xml").to_table()
    stars_wt_bestpara=ap.io.votable.parse_single_table("data/stars_wt_bestpara.xml").to_table()
    
    mesDist=query(TAP_service[0],upload_query[2],[stars_wt_bestpara[:].copy()])#needs stars without best para
    #join source_idref
    mesDist['bibcode']=mesDist['bibcode'].astype(str)
    mesDist=ap.table.join(mesDist,sources['ref','source_id'],keys_left='bibcode',keys_right='ref',join_type='left')
    mesDist.rename_column('source_id','mesDist_source_idref')
    save(mesDist,'data/mesDist')
    #group by object_idref
    grouped_mesDist=mesDist.group_by('object_idref')
    best_mesDist=mesDist['object_idref','dist','plus_err','qual','mesDist_source_idref'][:0]
    best_mesDist.rename_columns(['dist','plus_err','qual','mesDist_source_idref'],
                                ['dist_value','dist_err','dist_qual','dist_source_idref'])
    for i in range(len(grouped_mesDist.groups.keys)):
        #sort by quality
        row=grouped_mesDist.groups[i][np.where(grouped_mesDist['dist_prec'].groups[i]==np.max(grouped_mesDist['dist_prec'].groups[i]))][0]
        #take first and add to best_paras
        #which error to take when there are multiples...
        dist_err=max(row['plus_err'],-row['minus_err'])
        best_mesDist.add_row([row['object_idref'],row['dist'], dist_err,row['qual'], row['mesDist_source_idref']])
    #join with other multimes thingis
    best_paras=best_mesDist
    #best_paras=ap.table.join(best_mesDist,best_mesTemp,keys='object_idref',join_type=outer)
    #+same from other multimes tables but only one row per object
    save(best_paras,'data/best_paras')
    #problem: no quality entries
    #add to basik/replace in basic if there are multiple provider or provider not doing already
    return

#disks
#disks_table_creation()
#disks=ap.io.votable.parse_single_table("data/disks.xml").to_table()

#simbad
#query_simbad()
#simbad=ap.io.votable.parse_single_table("data/simbad.xml").to_table()


#stars
#stars_table_creation()
#stars=ap.io.votable.parse_single_table("data/stars.xml").to_table()


#exomercat
#query_exomercat(True)
#exomercat=ap.io.votable.parse_single_table("data/exomercat.xml").to_table()

#sources
#sources_table_creation()
#sources=ap.io.votable.parse_single_table("data/sources.xml").to_table()

#ident
#ident_table_creation()
#ident=ap.io.votable.parse_single_table("data/ident.xml").to_table()
#can I run this already with current exomercat data? how many exomercat objects do I include in that case?

#planets
#planets_table_creation()
#sim_planets=ap.io.votable.parse_single_table("data/sim_planets.xml").to_table()
#exo_planets=ap.io.votable.parse_single_table("data/exo_planets.xml").to_table()
#planets=ap.io.votable.parse_single_table("data/planets.xml").to_table()

#objects
#objects_table_creation()
#objects=ap.io.votable.parse_single_table("data/objects.xml").to_table()

#h_link
#h_link_table_creation()
#h_link=ap.io.votable.parse_single_table("data/h_link.xml").to_table()

#multimes
#multimeas_table_creation()
#mesDist=ap.io.votable.parse_single_table("data/mesDist.xml").to_table()
#best_paras=ap.io.votable.parse_single_table("data/best_paras.xml").to_table()

# important
I am in the middle of getting planets rewritten which includes changes in ident and h_link
I can actually adjust the code already without needing to wait for exomecat end of vacation as I have four planets in my volume cut.
I found out I could easily use simbad for joining the host identifiers, as the current problem is to get a match when only part of the identifier is identical e.g. '14 Her' vs '*  14 Her'

I still want to: 
- get exo_planets ready
- get sim_planets ready
- get planets ready, outer join on exo_planets and sim_planets
- get ident from planets (careful to get id_source_idref right for exo and sim planets
- get h_link ready

also... would it be faster to take ident table from stars oid upload instead of making the for loop?

In [ ]:
#how to match exo and sim planets? maybe also with a query on simbad to get the main_id

lets try if tap query with 14 Her works without *  
SELECT main_id, oid
FROM basic
JOIN ident ON oid = oidref
WHERE id='14 Her'
works

In [61]:
cat=simbad
print(cat)
print(cat.colnames)
print(cat[-1])

        main_id                 ra         ... type multiple
                               deg         ...              
----------------------- ------------------ ... ---- --------
         SCR J0723-8015 110.99834039126124 ...   st    False
            LP   66-542 213.71015319363957 ...   st    False
              HD 188807  299.3318437211008 ...   sy     True
             LP  779-34   93.7999345536804 ...   st    False
           BD+10  1857A 130.68553940910084 ...   sy     True
                    ...                ... ...  ...      ...
       UCAC4 193-187066         274.525341 ...   sy     True
              Ross  165         296.439537 ...   sy     True
              Ross  165         296.439537 ...   sy     True
2MASS J17072343-0558249         256.847647 ...   sy     True
          UCAC3 85-1639          11.434793 ...   sy     True
          UCAC3 85-1639          11.434793 ...   sy     True
Length = 3598 rows
['main_id', 'ra', 'dec', 'oid', 'coo_bibcode', 'plx_err', 'plx_val

# quality
simbad quality criterion position:
A 	standard object. Best quality data
B 	High quality data.
C 	Good quality. Values transformed into equinox and epoch 2000.0
D 	Values found in the litterature, without too much control. Typically data introduced through published papers.
E 	suspected value, for instance taken from an identifier

I will do something similar.
To create my quality parameter I will take into account the provider, quality flag given by provider, precission of the value and the error.
- provider
- provider quality flag
- value precission
- error

when making the decision of which measurement to include into the basic table I will take the better and in case of same quality will take the first.

this project requires to know which multi measurements we are interested in.
the own quality_flag creation is not my project, somebody else can do it and it might be needed to do so for each parameter where we include multiple measurements

In [ ]:
def quality_flag():
    # order providers according to trustworthness
    #better quality flag better than lower
    # high number of precission better than lower
    # low error/value better than higher one, higher one better than no error
    return

# optimization

In [ ]:
#define all the elements you want to put into source
sim_ref=[stars['coo_bibcode'],stars['plx_bibcode'],h_link['link_bibcode']]
exo_ref=[exomercat['bestmass_url']]
disk_ref=[disks['ref']]
provider_columns=[sim_ref,exo_ref,disk_ref]
cat_sources=ap.table.Table()

for i in range(len(provider_columns)):
    l=[]
    for col in provider_columns[i]:
        l+=col[np.where(col.mask==False)]
    temp=np.unique(l)
    cat_sources=ap.table.vstack([cat_sources,])
    #remove all the masked entries
#build lists from them for same provider
#take unique
np.unique(l)
#add provider info


## sorting input data in dictionary

In [23]:
#not urgent: sorting it
#make a dictionary out of this:
mydict =	{
  "object_name" : ['stars','planets'],
  "table": [stars,planets],
  "acquired_through": ['simbad','exomercat'],
  "file": ['stars.fits','planets.fits'],
  "link": ["http://simbad.u-strasbg.fr:80/simbad/sim-tap","http://archives.ia2.inaf.it/vo/tap/projects"]
}
#print(mydict)
#print(mydict["file"])
print(mydict["file"][0])#would be nice if I could call it by the object name. would need to find out how to get the place number from it.

stars.fits


## fits format

In [1]:
#ap fits stuff https://docs.astropy.org/en/stable/io/fits/index.html#save-file-changes
#ap.io.HDUList.writeto('stars.fits') #problem, what is saved is what is currently in memory. but as I will have multiple tables which of it will it take?
#stars.writeto('stars.fits') AttributeError: 'Table' object has no attribute 'writeto'
ap.io.fits.writeto('stars.fits',stars[:][:]) #'Data must be a numpy array.'
#...maybe it is easier to use votable in dachs than to figure out how to save a votable as fits
#votables can have multiple tables, ap.table.Table can have only one table.

In [ ]:
#not urgent: loading data
ap.io.fits.open('data/stars.fits')

When writing a file (e.g., with the writeto() function), compression will be determined based on the filename extension given, or the compression used in a pre-existing file that is being written to.

## add more info to ref

In [59]:
#not urgent: add more info to ref
#2. take only distinct of it
#table.unique(tablename, keys='columnname')
#3. join simbad ref info, do I really want that or not better the info about the acquired through?
#4. join simbad author info
source_query="""
SELECT bibcode, year
FROM ref
JOIN TAP_UPLOAD.t1
ON b.oid=t1.sim_parent_id
"""

In [ ]:
    for i in range(2):
        cat[i]=ap.table.vstack([init[i],sim[i]])
        cat[i]=ap.table.vstack([cat[i],gk[i]])
        cat[i]=ap.table.vstack([cat[i],exo[i]])
        if len(cat[i])>0:
            cat[i]=ap.table.unique(cat[i],silent=True)
            print('this way of adding objects together does not work')
            print('maybe I can add them using a join on main id')
            print('then merging ids1 and ids2 by first spliting them by |')
            print('then joining all ids as list, taking unique and merging with |')
        if i==0:#---------------sources------------------
            cat[i]['source_id']=[j+1 for j in range(len(cat[i]))]
        if i==1:#----------------objects----------------------------
            cat[i]['object_id']=[j+1 for j in range(len(cat[i]))]
            cat[i]=cat[1]['object_id','type','ids','main_id']

In [52]:
catalog_names=['stars','planets','h_link','disks']

#pre h_link code
catalog = [ap.table.Table() for i in range(len(catalog_names))]
catalog_sources=[ap.table.Table() for i in range(len(catalog_names))]


    
#do the same for the stars and the planets objects
for i in range(len(catalog_names)):
    #Performs a query via TAP on the service. 
    service = vo.dal.TAPService(TAP_service[i])
    result=service.run_async(adql_query[i].format(**locals()), maxrec=160000)
    catalog[i]=result.to_table()
    #creates a source table from the references given in the ref_columns
    catalog_reflist=[]
    for j in ref_columns[i]:
        catalog_reflist.extend(catalog[i][j])
    catalog_sources[i]['ref']=catalog_reflist
    catalog_sources[i]=ap.table.unique(catalog_sources[i])#keeps only unique values
    #attaches service information
    catalog_sources[i]['provider_name']=[provider_name[i]]*len(catalog_sources[i])
    catalog_sources[i]['provider_url']=[TAP_service[i]]*len(catalog_sources[i])
    catalog_sources[i]['provider_bibcode']=[provider_bibcode[i]]*len(catalog_sources[i])
sources=ap.table.vstack([catalog_sources[0],catalog_sources[1]])
sources=ap.table.vstack([sources,catalog_sources[2]])
sources['source_id']=[i+1 for i in range(len(sources))]#introduces an identifier column

In [ ]:
l_pl.append("Pl" in cat["otypes"][i])
        l_sy.append("**" in cat["otypes"][i])
        l_st.append("*" in cat["otypes"][i])
    cat['type']=cat[np.where(l_pl)]
    systems=cat[np.where(l_sy)]
    l_st=[(st and not(st and sy)) for st,sy in zip(l_st,l_sy)]#[true,false,true,false,true,true] operator [false,true,true,true,false,true] gives [true,false,false,false,true,false]
    l_st=[(st and not(st and pl)) for st,pl in zip(l_st,l_pl)]#[true,false,true,false,true,true] operator [false,true,true,true,false,true] gives [true,false,false,false,true,false]
    stars=cat[np.where(l_st)]#would like to remove the system and planet ones

In [530]:
#go through exomercat entries
n=len(exomercat)+1
for i_sim in range(len(sim_planets)):
    for i_exo in range(len(exomercat)):
        for simids in sim_planets['ids'][i_sim].split('|'):
            if simids in exomercat['ids'][i_exo].split('|'):
                sim_planets['object_idref'][i_sim]=exomercat['object_idref'][i_exo]
    if sim_planets['object_idref'][i_sim]=='0000':
        sim_planets['object_idref'][i_sim]='P%04d' % n
        n+=1
#lenrest=len(sim_planets['object_idref'][np.where(sim_planets['object_idref']=='0000')])  
#oids=['P%04d' % (j+1) for j in range(len(exomercat),len(exomercat)+lenrest)]
#sim_planets['object_idref'][np.where(sim_planets['object_idref']=='0000')]=oids

#does not work as it changes with time


#I have this issue that I have to loop over multiple catalogs, isn't there a better way?
                
            
#go through simbad entries
#match if any of the simbad ids is in one of the exomercat ids
#if yes give simbad entry same object_idref, otherwise give new one

#create ident from those two tables.
#merge exomercat and simbad into one table by adding ids if same object_idref and keeping oid
#planets=ap.table.vstack([sim_planets,exomercat['ids','bestmass','bestmass_url','type']])

/tmp/ipykernel_110994/2584723597.py:21: StringTruncateWarning: truncated right side string(s) longer than 4 character(s) during assignment
  sim_planets['object_idref'][i_sim]='P%04d' % n
/tmp/ipykernel_110994/2584723597.py:21: StringTruncateWarning: truncated right side string(s) longer than 4 character(s) during assignment
  sim_planets['object_idref'][i_sim]='P%04d' % n
/tmp/ipykernel_110994/2584723597.py:21: StringTruncateWarning: truncated right side string(s) longer than 4 character(s) during assignment
  sim_planets['object_idref'][i_sim]='P%04d' % n
/tmp/ipykernel_110994/2584723597.py:21: StringTruncateWarning: truncated right side string(s) longer than 4 character(s) during assignment
  sim_planets['object_idref'][i_sim]='P%04d' % n
/tmp/ipykernel_110994/2584723597.py:21: StringTruncateWarning: truncated right side string(s) longer than 4 character(s) during assignment
  sim_planets['object_idref'][i_sim]='P%04d' % n
/tmp/ipykernel_110994/2584723597.py:21: StringTruncateWarnin

/tmp/ipykernel_110994/2584723597.py:21: StringTruncateWarning: truncated right side string(s) longer than 4 character(s) during assignment
  sim_planets['object_idref'][i_sim]='P%04d' % n
/tmp/ipykernel_110994/2584723597.py:21: StringTruncateWarning: truncated right side string(s) longer than 4 character(s) during assignment
  sim_planets['object_idref'][i_sim]='P%04d' % n
/tmp/ipykernel_110994/2584723597.py:21: StringTruncateWarning: truncated right side string(s) longer than 4 character(s) during assignment
  sim_planets['object_idref'][i_sim]='P%04d' % n
/tmp/ipykernel_110994/2584723597.py:21: StringTruncateWarning: truncated right side string(s) longer than 4 character(s) during assignment
  sim_planets['object_idref'][i_sim]='P%04d' % n
/tmp/ipykernel_110994/2584723597.py:21: StringTruncateWarning: truncated right side string(s) longer than 4 character(s) during assignment
  sim_planets['object_idref'][i_sim]='P%04d' % n
/tmp/ipykernel_110994/2584723597.py:21: StringTruncateWarnin

In [417]:
#I believe I need to do ident first, otherwise I loose provider info of id
#issue with that is, that for ident I need object_id which means I will have double entries...
#except if I use oid as temporarely object_id and replace with object_id if there are matches or create new object_id else

l=[]
for i_cat2 in range(len(cat2)):
    for i_cat1 in range(len(cat1)):
        #later make a for here too to get all cat1 ids elements checked
        for cat1ids in cat1['ids'][i_cat1].split('|'):
            if cat1ids in cat2['ids'][i_cat2].split('|'):
                print('double object',cat1['ids'][i_cat1].split('|')[0])
                l.append([i_cat1,i_cat2])
                #merging cat.add_row(cat1['ids'][i_cat1]+'|'+cat2['ids'][i_cat2])
                #how do I deal with the fact that multiple ids of cat 1 will be in cat1 when they are double objects?
                #need to get out of this for look if the if is true
#remove cat1 rows here l not zero
#cat is vstack of cat1 and cat2 except where l not null ids of becomes cat1['ids'][i_cat1]+'|'+cat2['ids'][i_cat2]

#no object printed as double object, so lets if that means there is none because of small sample size or it is not working: see if it works if I fake one
#fake did work, made cat2['ids'][i_cat2].split('|')[0] in cat2['ids'][i_cat2].split('|')
#now query more exomercat -> service still offline
#see if it helps if I query whole simbad -> no did not get any
#try going through all cat1 ids -> no result

In [259]:
#h_link oid is not unique as there are multiple membership entries usually
#takes a bit more than a minute
#linking objects with their parents, transforming oid into object_id
h_link['child']=['00000' for j in range(len(h_link))]
h_link['parent_object_id']=['00000' for j in range(len(h_link))]

#replaces oid in column child with object_idre of object table if oid of h_link same as oid of object table
#replaces 00000 in column parent_object_id with object_idre of object table if oid in parent column of h_link same as oid of object table
for i_hlink in range(len(h_link)):
    #go through all param bibcode entries in stars
    for i_objects in range(len(objects)):
        #go through all ref entries in sources
        if h_link['oid'][i_hlink]==objects['oid'][i_objects]:
            h_link['child'][i_hlink]=objects['object_id'][i_objects]
        if h_link['parent'][i_hlink]==objects['oid'][i_objects]:
            h_link['parent_object_id'][i_hlink]=objects['object_id'][i_objects]
#there are quite some objects that do have a parent which we will not be interested in (e.g. moving group.)
h_link=h_link[np.where(h_link['parent_object_id']!='00000')]#taking them out

In [ ]:
def idref(cat,oldcolnames,newcolnames,sources):
    """
    For each entry in one of the oldcolnames columns in table cat,
    looks up the corresponding source identifier source_id in the 
    source table and adds it in the column <newcolname>_source_idref. Deletes 
    the columns of oldcolnames.
    :param cat: astropy table
    :param oldcolnames: list of strings naming the columns in cat where the reference entries are
    :param newcolnames: list containing the strings to be added at the front of the 
        colmnname _source_idref.
    :return cat: astropy table
    """
#I might be able to do this as a join

#go over all reference parameter columns
    for k in range(len(oldcolnames)):
        if f'{newcolnames[k]}_source_idref' in cat.colnames:
            cat.remove_column(f'{newcolnames[k]}_source_idref')
        #here I get an error message
        cat[oldcolnames[k]]=cat[oldcolnames[k]].astype(np.str_)
        if type(cat[oldcolnames[k]])==ap.table.column.MaskedColumn:
            cat_unmasked=cat[np.where(cat[oldcolnames[k]].mask==False)]
            cat_masked=cat[np.where(cat[oldcolnames[k]].mask==True)]
            cat_unmasked=ap.table.join(cat_unmasked,sources['ref','source_id'],keys_left=oldcolnames[k],keys_right='ref',join_type='left')
            cat=ap.table.vstack([cat_unmasked,cat_masked])
        else:
            cat=ap.table.join(cat,sources['ref','source_id'],keys_left=oldcolnames[k],keys_right='ref',join_type='left')
        cat.remove_column('ref')
        cat.rename_column('source_id',f'{newcolnames[k]}_source_idref')
    return cat

In [ ]:
def query_simbad(temp=False):
    """
    This function queries from the SIMBAD database all the 
    objects within 20pc of the Sun. It obtains for each the entries
    for the parameters main identifier, right ascencion, declination,
    object internal identifier, bibcode of the coordinate parameter, 
    parallax value, parallax error, parallax bibcode, object internal
    identifier of the parent object, likelyhood of the parent relation,
    bibcode of the parent relation, object types and identifiers. It 
    then sorts the objects into star, system and planet types. Finally
    the table is saved.
    """
    #queries SIMBAD for objects with parallax value > 50 mas
    TAP_service="http://simbad.u-strasbg.fr:80/simbad/sim-tap"
    simbad=query(TAP_service,adql_query[0])
    #adds parent and children objects with no parallax value
    if temp: #as long as simbad table upload is not working via this temporary query
        tempquery1="""
        SELECT b.main_id,b.ra,b.dec,b.oid, b.coo_bibcode, b.plx_err, b.plx_value, b.plx_bibcode,
        h_link.membership , h_link.parent, h_link.link_bibcode, a.otypes,i.ids
        FROM basic AS b
        JOIN ids AS i ON b.oid=i.oidref
            JOIN alltypes AS a ON b.oid=a.oidref
                LEFT JOIN h_link ON b.oid=h_link.child
                    JOIN (
                        SELECT b0.main_id,b0.ra,b0.dec,b0.oid, b0.coo_bibcode, b0.plx_err, b0.plx_value, b0.plx_bibcode,
                        h.membership , h.parent, h.link_bibcode, a0.otypes,i0.ids 
                        FROM basic AS b0
                        JOIN ids AS i0 ON i0.oidref=b0.oid
                            JOIN alltypes AS a0 ON b0.oid=a0.oidref
                                LEFT JOIN h_link AS h ON b0.oid=h.child
                        WHERE b0.plx_value >=50.) AS t1 ON b.oid=t1.parent
        WHERE (b.plx_value IS NULL) AND (otype='**..') 

        """
        parents_without_plx=query(TAP_service,tempquery1)#one object
        tempquery2="""
        SELECT b.main_id,b.ra,b.dec,b.oid, b.coo_bibcode, b.plx_err, b.plx_value, b.plx_bibcode,
        h_link.membership , h_link.parent, h_link.link_bibcode, a.otypes,i.ids
        FROM basic AS b
        JOIN ids AS i ON b.oid=i.oidref
            JOIN alltypes AS a ON b.oid=a.oidref
                LEFT JOIN h_link ON b.oid=h_link.child
                    JOIN (
                        SELECT b0.main_id,b0.ra,b0.dec,b0.oid, b0.coo_bibcode, b0.plx_err, b0.plx_value, b0.plx_bibcode,
                        h.membership , h.parent, h.link_bibcode, a0.otypes,i0.ids 
                        FROM basic AS b0
                        JOIN ids AS i0 ON i0.oidref=b0.oid
                            JOIN alltypes AS a0 ON b0.oid=a0.oidref
                                LEFT JOIN h_link AS h ON b0.oid=h.child
                        WHERE b0.plx_value >=50.) AS t1 ON b.oid=t1.oid
        WHERE (b.plx_value IS NULL) AND (otype='Pl..')

        """
        children_without_plx=query(TAP_service,tempquery2)#no object
    else:
        parents_without_plx=query(TAP_service,upload_query[0],[simbad])
        children_without_plx=query(TAP_service,upload_query[1],[simbad])
    #adding of no_parallax objects to rest of simbad query objects
    simbad=ap.table.vstack([simbad,parents_without_plx])
    simbad=ap.table.vstack([simbad,children_without_plx])
    #sorting from object type into star, system and planet type
    simbad=sorting_otypes(simbad)
    #saving table
    ap.io.votable.writeto(ap.io.votable.from_table(simbad),
                          f'data/simbad.xml')
    return

In [246]:
#checking if best_paras dist_err is bigger error of mesDist plus and minus

#temp=mesDist['plus_err']+mesDist['minus_err']
#print(temp[np.where(temp!=0)])
print(mesDist['object_idref','plus_err','minus_err'][np.where(mesDist['plus_err']+mesDist['minus_err']!=0)])
#row['diff']=row['plus_err']+row['minus_err']
#print(row[np.where(row['diff']!= 0.)])
print(mesDist['object_idref','plus_err','minus_err'][np.where(mesDist['object_idref']=='S2096')])
print(best_paras['dist_err'][np.where(best_paras['object_idref']=='S2096')])

object_idref plus_err minus_err
------------ -------- ---------
       S0619       --        --
       S0008       --        --
       S1457       --        --
       S1623       --        --
       S1688       --        --
       S0115       --        --
         ...      ...       ...
       S2467     0.44      0.39
       S2524     0.25      0.23
       S2876      1.5       1.2
       S2810       --        --
       S2096       --      -5.0
       S2096       --      -5.0
Length = 1045 rows
object_idref plus_err minus_err
------------ -------- ---------
       S2096   0.0003   -0.0003
       S2096       --      -5.0
       S2096       --      -5.0
dist_err
--------
  0.0003


In [ ]:
def sources_table_creation(temp=False):
    """
    This function creates the provenance information by first reading
    in the stars, exomercat, simbad, disks and mesDist data obtained
    through the similarly named functions. The h_link table gets defined
    as a subset of the simbad one by removing rows with parent relation
    probability smaller than 50%. From all those tables the reference 
    column entries are stored in the source table. Columns for provider
    name, provider bibcode and provider url are added together with
    a source identifier.
    """
    stars=ap.io.votable.parse_single_table("data/stars.xml").to_table()
    exomercat=ap.io.votable.parse_single_table("data/exomercat.xml").to_table()
    simbad=ap.io.votable.parse_single_table("data/simbad.xml").to_table()
    disks=ap.io.votable.parse_single_table("data/disks.xml").to_table()
    mesDist=ap.io.votable.parse_single_table("data/mesDist.xml").to_table()
    sources=ap.table.Table()
    h_link=simbad['oid','membership','parent','link_bibcode',
                  'otypes','multiple'][np.where(simbad['membership']>50)]
    
    if temp:
        ref_columns=[['coo_bibcode','plx_bibcode'],['bestmass_url'],['link_bibcode'],['disks_ref']]
        provider_name=['SIMBAD','Exo-MerCat','SIMBAD','None']
        provider_bibcode=['2000A&AS..143....9W','2020A&C....3100370A','2000A&AS..143....9W','None']
        TAP_service=["http://simbad.u-strasbg.fr:80/simbad/sim-tap",
            "http://archives.ia2.inaf.it/vo/tap/projects","http://simbad.u-strasbg.fr:80/simbad/sim-tap","None"]
        for i,cat in zip(range(4),[stars,exomercat,h_link,disks]):
            sources=sources_table(cat,ref_columns[i],[provider_name[i],TAP_service[i],provider_bibcode[i]],sources)
            #add row for provider data without producer reference in case of simbad and exomercat
            if i in [0,1]:
                sources.add_row(['',provider_name[i],TAP_service[i],provider_bibcode[i]])
    else:
        TAP_service=["http://simbad.u-strasbg.fr:80/simbad/sim-tap",
            "http://archives.ia2.inaf.it/vo/tap/projects","None"]
        provider_name=['SIMBAD','Exo-MerCat','None']
        provider_bibcode=['2000A&AS..143....9W','2020A&C....3100370A','None']
        ref_columns=[
            [['coo_bibcode','plx_bibcode'],['link_bibcode'],['bibcode']],#simbad queried reference columns
                     [['bestmass_url']],#exomercat queried reference columns
                     [['disks_ref']]]#no provider reference columns
        tables=[[stars, h_link, mesDist],
                [exomercat],
                [disks]]
        for i in range(3):
            for cat,ref in zip(tables[i],ref_columns[i]):
                sources=sources_table(cat,ref,[provider_name[i],TAP_service[i],provider_bibcode[i]],sources)
            if i != 2:
                sources.add_row(['',provider_name[i],TAP_service[i],provider_bibcode[i]])
    
    sources['source_id']=[j+1 for j in range(len(sources))]#introduces an identifier column
    save(sources,'data/sources')
    return

In [ ]:
def sorting_otypes(cat): #put this into simbad function
        cat['type']=['None' for i in range(len(cat))]
        cat['multiple']=[False for i in range(len(cat))]
        for i in range(len(cat)):
            if "Pl" in cat['otypes'][i]:
                cat['type'][i]='pl'
            elif "*" in cat['otypes'][i]:
                if "**" in cat['otypes'][i]:
                    cat['type'][i]='sy'
                    cat['multiple'][i]=True
                else:
                    cat['type'][i]='st'
            else:
                print('Neither Pl,* or **:',cat['otypes'][i])
        return cat

In [ ]:
def update_database_tables(new,table):
    """
    preferably do sources befor any other tables
    :param update: list of astropy tables corresponding
    """
    [sources,objects,ident,h_link,star_basic,planet_basic,disk_basic,mesDist]=load(['sources','objects','ident',
                                    'h_link','star_basic','planet_basic','disk_basic','mesDist'])
    
    if table=='h_link':
        h_link=ap.table.vstack([new,h_link])
        h_link=ap.table.unique(h_link,silent=True)
        h_link.remove_columns(['child_object_idref','parent_object_idref'])
        h_link=ap.table.join(h_link,objects['main_id','object_id'],join_type='left')
        h_link.rename_column('object_id','child_object_idref')
        h_link.remove_column('main_id')
        h_link=ap.table.join(h_link,objects['main_id','object_id'],
                             keys_left='parent_main_id',keys_right='main_id')
        h_link.rename_column('object_id','parent_object_idref')
        h_link.remove_columns(['main_id','parent_main_id'])
    if table=='mesDist':
        mesDist=ap.table.vstack([new,mesDist])
        mesDist=ap.table.unique(mesDist,silent=True)
        mesDist.remove_column('object_idref')
        mesDist=ap.table.join(mesDist,objects['main_id','object_id'],join_type='left')
        mesDist.rename_column('object_id','object_idref')
        mesDist.remove_column('main_id')
    if table=='planet_basic':
        planet_basic=ap.table.vstack([new,planet_basic])
        planet_basic=ap.table.unique(planet_basic,keys='main_id')
        planet_basic.remove_column('object_idref')
        planet_basic=ap.table.join(planet_basic,objects['main_id','object_id'],join_type='left')
        planet_basic.rename_column('object_id','object_idref')
        planet_basic.remove_column('main_id')
    if table=='star_basic':
        star_basic=ap.table.vstack([new,star_basic])
        star_basic=ap.table.unique(star_basic,silent=True)
        star_basic.remove_column('object_idref')
        star_basic=ap.table.join(star_basic,objects['main_id','object_id'],join_type='left')
        star_basic.rename_column('object_id','object_idref')
        star_basic.remove_column('main_id')
    
    if table=='ident':
        ident=ap.table.vstack([new,ident])
        ident=ap.table.unique(ident,silent=True)
        ident.remove_column('object_idref')
        ident=ap.table.join(ident,objects['main_id','object_id'],join_type='left')
        ident.rename_column('object_id','object_idref')
        ident.remove_column('main_id')
    if table=='objects':
        #add main id to other tables
        
        #then update object
        objects=ap.table.vstack([new,objects])
        objects=ap.table.unique(objects,silent=True)
        objects['object_id']=[j+1 for j in range(len(objects))]
        objects=objects['object_id','type','ids','main_id']#changing order of columns
        #then update object_idref
        #---------now outer join object_idref sorted by type to _basic to show missing data
        planet_basic.remove_column('object_idref')
        planet_basic=ap.table.join(planet_basic,objects['main_id','object_id'],join_type='outer')
        planet_basic.rename_column('object_id','object_idref')
    
    def match(cat,sources,paras,idref=True):
        for para in paras:
            if idref:
            #problem, join does not work with empty table need 
                if len(cat)!=0 and len(sources)!=0:
                    if type(cat[para+'_source_idref'])==ap.table.column.MaskedColumn:
                        cat[para+'_source_idref'].fill_value=0
                        cat=cat.filled()
                    cat=ap.table.join(cat,sources['ref','source_id'],
                            keys_left=para+'_source_idref',keys_right='source_id',join_type='left')
                    cat.rename_column('ref',f'{para}_ref')
                    cat.remove_column('source_id')
            else:
                if para+'_ref' in cat.colnames:
                    if type(cat[para+'_ref'])==ap.table.column.MaskedColumn:
                        cat[para+'_ref'].fill_value=''
                        cat=cat.filled()
                    cat.remove_column(f'{para}_source_idref')
                    cat=ap.table.join(cat,sources['ref','source_id'],keys_left=para+'_ref',keys_right='ref',join_type='left')
                    cat.rename_column('source_id',f'{para}_source_idref')
                    cat.remove_columns(['ref',para+'_ref'])
                    #in case the para_value entry is masked this if environment will put the source_id entry to null
                    if para+'_value' in cat.colnames:
                        if type(cat[para+'_value'])==ap.table.column.MaskedColumn:
                            for i in cat[para+'_value'].mask.nonzero()[0]:
                                cat[f'{para}_source_idref'][i]=0
        return cat
    
    
    if table=='sources':
    #----------------check for duplicates--------
        
        #add refs to other tables
        
        ident=match(ident,sources,['id'])
        h_link=match(h_link,sources,['h_link'])
        star_basic=match(star_basic,sources,['coo','plx','dist'])
        planet_basic=match(planet_basic,sources,['mass'])
        disk_basic=match(disk_basic,sources,['rad'])
        mesDist=match(mesDist,sources,['dist'])
            
        sources=ap.table.vstack([new,sources])
    #ref in sources unique
        sources=ap.table.unique(sources,silent=True)
    #main_id unique
    #--------------add data----------------
    #-----------------update foreign keys-------------
    ###-------------------update source id-------------
        sources['source_id']=[j+1 for j in range(len(sources))]#introduces an identifier column

    ###-------------------update source id ref------------
    #ident id_source_idref, h_link h_link_source_idref, star_basic coo_source_idref, plx_source_idref, 
    #planet_basic mass_source_idref, disk_basic rad_source_idref, mesDist dist_source_idref
    #now at the start there are no columns para+'_bibcode' in the tables,
    #wait, but when I load the old tables I will need to redo the source_idref and therefore need the para_ref
    #so I do need to save those in the tables themselves or obtain them from old sources table
        
    if len(sources)==0:
        print('please first ingest some sources data')
    else:
        ident=match(ident,sources,['id'],False)
        h_link=match(h_link,sources,['h_link'],False)
        star_basic=match(star_basic,sources,['coo','plx','dist'],False)
        planet_basic=match(planet_basic,sources,['mass'],False)
        disk_basic=match(disk_basic,sources,['rad'],False)
        mesDist=match(mesDist,sources,['dist'],False)
    
    
    ###-------------------update object id(ref)----------------
    #I can easily write this in a for loop though that would take longer than np.where assignment
    #np.where assignment issue
    

    #stars['object_idref']=['S%04d' % (j+1) for j in range(len(stars))]

    ###-------------------update parent-----------------
    
    save([sources,objects,ident,h_link,star_basic,planet_basic,disk_basic,mesDist],
         ['sources','objects','ident','h_link','star_basic','planet_basic','disk_basic','mesDist'])
    return sources,objects,ident,h_link,star_basic,planet_basic,disk_basic,mesDist

# update tables

In [ ]:
n=0
cats=[sources,objects,ident,h_link,star_basic,planet_basic,mesDist]
sim_new=[sim_sources,sim_objects,sim_ident,sim_h_link,sim_star_basic,sim_mesDist]
gk_new=[gk_sources,gk_objects, gk_ident, gk_h_link,gk_disk_basic]
table=['sources','objects','ident','h_link','star_basic',
          'planet_basic','mesDist']

[sources,objects,ident,h_link,star_basic,planet_basic,disk_basic,mesDist]=update_database_tables(
    sim_new[n],table[n])